# Energy Fraud Detection

Imports for notebook:

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import seaborn as sns
%matplotlib inline

#from itables import init_notebook_mode
#init_notebook_mode(all_interactive=True)

## Introduction

Looking at dataset of clients and their energy usage over time. The target variable is fraud, and is labelled on a per-client basis.

This is an interesting problem where data is partially corrupted, and has major inconsistencies. Furthermore, there is insufficient information to obtain target variable for all cases.

This notebook conducts the initial data cleaning, where the raw datasets are input, and the 'cleaned' datasets are output.

When rows are removed, they are placed in the df_removed_{train|test} dataset. At least one row for every 'client_id' is needed.

Where relevant, the last row will be kept for the 'client_id' if all would otherwise be removed.

Please refer to the main notebook, "???", for further details and continuation.

## Data Cleaning

### Data Import

In [2]:
# Read the CSV files
df_client_test = pd.read_csv('./client_test.csv', on_bad_lines='skip')
df_client_train = pd.read_csv('./client_train.csv', on_bad_lines='skip')
df_invoice_test = pd.read_csv('./invoice_test.csv', on_bad_lines='skip')
# low_memory is prompted due to unexpected values and large datasize
df_invoice_train = pd.read_csv('./invoice_train.csv', on_bad_lines='skip', low_memory=False)
df_SampleSubmission = pd.read_csv('./SampleSubmission (2).csv', on_bad_lines='skip')

In [3]:
df_SampleSubmission.head()

,client_id,target
0,test_Client_0,0.957281
1,test_Client_1,0.996425
2,test_Client_10,0.612359
3,test_Client_100,0.776933
4,test_Client_1000,0.571046


In [4]:
df_client_test.head()

,disrict,client_id,client_catg,region,creation_date
0,62,test_Client_0,11,307,28/05/2002
1,69,test_Client_1,11,103,06/08/2009
2,62,test_Client_10,11,310,07/04/2004
3,60,test_Client_100,11,101,08/10/1992
4,62,test_Client_1000,11,301,21/07/1977


In [5]:
df_invoice_test.head()

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,consommation_level_4,old_index,new_index,months_number,counter_type
0,test_Client_0,2018-03-16,11,651208,0,203,8,1,755,0,0,0,19145,19900,8,ELEC
1,test_Client_0,2014-03-21,11,651208,0,203,8,1,1067,0,0,0,13725,14792,8,ELEC
2,test_Client_0,2014-07-17,11,651208,0,203,8,1,0,0,0,0,14792,14792,4,ELEC
3,test_Client_0,2015-07-13,11,651208,0,203,9,1,410,0,0,0,16122,16532,4,ELEC
4,test_Client_0,2016-07-19,11,651208,0,203,9,1,412,0,0,0,17471,17883,4,ELEC


In [6]:
print(f"Number of rows in client train vs invoice train: {len(df_client_train)} vs {len(df_invoice_train)}")
print(f"Number of unique client_id in client train vs invoice train: {df_client_train['client_id'].nunique()} vs {df_invoice_train['client_id'].nunique()}")
print(f"Number of rows in client test vs invoice test: {len(df_client_test)} vs {len(df_invoice_test)}")
print(f"Number of unique client_id in client test vs invoice test: {df_client_test['client_id'].nunique()} vs {df_invoice_test['client_id'].nunique()}", end="")

Number of rows in client train vs invoice train: 135493 vs 4476749
Number of unique client_id in client train vs invoice train: 135493 vs 135493
Number of rows in client test vs invoice test: 58069 vs 1939730
Number of unique client_id in client test vs invoice test: 58069 vs 58069

Going to merge df_client_train and df_invoice_train:

In [7]:
df_test = df_invoice_test.join(df_client_test.set_index('client_id'), on='client_id', validate='m:1').copy()
df_train = df_invoice_train.join(df_client_train.set_index('client_id'), on='client_id', validate='m:1').copy()
df_train

,client_id,invoice_date,tarif_type,counter_number,counter_statue,counter_code,reading_remarque,counter_coefficient,consommation_level_1,consommation_level_2,...,consommation_level_4,old_index,new_index,months_number,counter_type,disrict,client_catg,region,creation_date,target
0,train_Client_0,2014-03-24,11,1335667,0,203,8,1,82,0,...,0,14302,14384,4,ELEC,60,11,101,31/12/1994,0.0
1,train_Client_0,2013-03-29,11,1335667,0,203,6,1,1200,184,...,0,12294,13678,4,ELEC,60,11,101,31/12/1994,0.0
2,train_Client_0,2015-03-23,11,1335667,0,203,8,1,123,0,...,0,14624,14747,4,ELEC,60,11,101,31/12/1994,0.0
3,train_Client_0,2015-07-13,11,1335667,0,207,8,1,102,0,...,0,14747,14849,4,ELEC,60,11,101,31/12/1994,0.0
4,train_Client_0,2016-11-17,11,1335667,0,207,9,1,572,0,...,0,15066,15638,12,ELEC,60,11,101,31/12/1994,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4476744,train_Client_99998,2005-08-19,10,1253571,0,202,9,1,400,135,...,0,3197,3732,8,ELEC,60,11,101,22/12/1993,0.0
4476745,train_Client_99998,2005-12-19,10,1253571,0,202,6,1,200,6,...,0,3732,3938,4,ELEC,60,11,101,22/12/1993,0.0
4476746,train_Client_99999,1996-09-25,11,560948,0,203,6,1,259,0,...,0,13884,14143,4,ELEC,60,11,101,18/02/1986,0.0
4476747,train_Client_99999,1996-05-28,11,560948,0,203,6,1,603,0,...,0,13281,13884,4,ELEC,60,11,101,18/02/1986,0.0


In [8]:
del df_invoice_train, df_invoice_test, df_client_train, df_client_test, df_SampleSubmission

### Renaming Columns and setting Data Types

There are some comments from the source regarding the meaning of these columns. These included here verbatim.
* "client_train.csv":
* "disrict: District where the client is"
* "client_id: Unique id for client"
* "client_catg: Category client belongs to"
* "region: Area where the client is"
* "creation_date: Date client joined"
* "target: fraud:1, not fraud: 0"
* "invoice_train.csv":
* "client_id: Unique id for client"
* "invoice_date: Date of the invoice"
* "tarif_type: Type of tax"
* "counter_number: number"
* "counter_statue: akes up to 5 values such as working fine, not working, on hold statue, ect"
* "counter_code: code"
* "reading_remarque: notes that the STEG agent takes during his visit to the cleint (e.g.: if the counter shows something wrong, the"
* "counter_coefficient: An additional coefficient to be added when standard consumption is exceeded"
* "consommation_level_1: Consumption_level_1"
* "consommation_level_2: Consumption_level_2"
* "consommation_level_3: Consumption_level_3"
* "consommation_level_4: Consumption_level_4"
* "old_index: Old index"
* "new_index: New index"
* "months_number: Month number"
* "counter_type: Type of counter"

Going to rename the columns slightly to better match my interpretation of the data gained from inspection.

In [9]:
rename_dict = {'client_id' : 'client_id', 
               'invoice_date' : 'invoice_date', 
               'tarif_type' : 'mtr_tariff', 
               'counter_number' : 'mtr_id',
               'counter_statue' : 'mtr_status', 
               'counter_code' : 'mtr_code', 
               'reading_remarque' : 'mtr_notes',
               'counter_coefficient' : 'mtr_coef', 
               'consommation_level_1' : 'usage_1', 
               'consommation_level_2' : 'usage_2',
               'consommation_level_3' : 'usage_3', 
               'consommation_level_4' : 'usage_4', 
               'old_index' : 'mtr_val_old',
               'new_index' : 'mtr_val_new', 
               'months_number': 'months_num', 
               'counter_type' : 'mtr_type', 
               'disrict' : 'district', 
               'client_catg' : 'client_type',
               'region' : 'region', 
               'creation_date' : 'start_date', 
               'target' : 'fraud'}

df_test.rename(columns=rename_dict, inplace=True) # Note that test won't have target
df_train.rename(columns=rename_dict, inplace=True)

Converting data types where appropriate:

In [10]:
# invoice_date: object -> date [YYYY-MM-DD] -> [YYYY-MM-DD]
df_train['invoice_date'] = pd.to_datetime(df_train['invoice_date'])
# start_date: object -> date [DD/MM/YYYY] -> [YYYY-MM-DD]
df_train['start_date'] = pd.to_datetime(df_train['start_date'], dayfirst=True)

col_names = ['mtr_type', 'district', 'client_type', 'region']
df_train[col_names] = df_train[col_names].astype("category")

# Converting 'mtr_val_' into float for now, as decimal places get in the way otherwise
df_train['mtr_val_old'] = df_train['mtr_val_old'].astype(float).round(0)
df_train['mtr_val_new'] = df_train['mtr_val_new'].astype(float).round(0)

### Data Structure and Hierarchy
Please refer to main notebook for the context. There is an exception in relation to 'mtr_tariff' == 9, where it interacts with coinciding 'mtr_tariff' == 11. This will be handled explictly later, however, a column will be made now for future population.

In [20]:
# This will change depending on context, but is the generic grouping to order rows.
grp_cols = ['client_id', 'mtr_type', 'mtr_tariff', 'mtr_id'] # Then to be sorted by: 'invoice_date'
df_train['temp_1'] = np.nan # Just for calculating and flagging temporarily 
df_train['temp_2'] = np.nan # Just for calculating and flagging temporarily 
df_train['usage_aux'] = 0 # This will house relevant usage from neighbouring invoices
df_train['monthly_usage_aux'] = np.nan # above scaled by invoice span
df_train['usage_aux_flag'] = 0 # Will indicate if being used

#### "Deep" sorting
There is a complication in that very rarely, this grouping, for a given 'invoice_date', may contain two 'mtr_code'. In this case, the ordering should be to try and match 'mtr_code' to the neighbouring rows.

In the case where neither row is matching neighbouring rows' 'mtr_code', they are reversed. the 'Calc_Sort()' function contains the grouping / sorting operation. Again, there is an exception in relation to 'mtr_tariff' == 9, which will be handled later.

In [12]:
def Calc_Sort(df_train, col_name):
    df_train.sort_values(by=grp_cols+['invoice_date'], inplace=True)
    df_train['temp_flag'] = np.nan
    df_temp = df_train.groupby(grp_cols, observed=True)
    df_train['temp_flag'] = ((df_train['mtr_code'] != df_temp['mtr_code'].shift(1)) & (df_train['mtr_code'] != df_temp['mtr_code'].shift(-1))).astype(int) #prev
    df_train['temp_flag'] = df_temp['temp_flag'].transform('cumsum') *-1
    df_train.sort_values(grp_cols + (['invoice_date', 'temp_flag'] if col_name is None else ['invoice_date', 'temp_flag', col_name]), inplace=True) # usage_n will be added here once calculated
    return df_train
# This will need to be updated as aspects such dates change.
df_train = Calc_Sort(df_train.copy(), None)

Based on data mining, some specific relationships have been deduced:
* usage_1 -> usage_4 are sequential "buckets" of usage, that if capped, can "spill" over into the next level.
* It is unclear how the presence of a cap is determined:
  * If mtr_tariff == [10|11] & mtr_code != [3__], more than usage_1 may be used.
  * If usage_3 is being used, so will usage_4.
  * usage_3 cap is equal to usage_1 cap. usage_2 cap is equal half usage_1 cap. usage_4 is uncapped.
  * Order of use is sequential; i.e., cannot use usage_2 before usage_1, etc.
  * If monthly cap == [50 | 300], usage_3 & usage_4 are not used.
* It is unclear how the quantity of the cap is determined:
  * The mtr_tariff and mtr_code seem to have an influence. mtr_status also seems to be influential.
  * If mtr_tariff == [10] & mtr_code == [1__], monthly cap is typically 50 | 100.
  * If mtr_tariff == [10] & mtr_code == [2__], monthly cap is typically 50 | 200.
  * If mtr_tariff == [11], monthly cap is typically 200 | 300.
* The "monthly cap" would be multiplied by the "months_num" to give the cap. This applies most of the time but cannot be relied upon.
* It is being assumed that there is an allowance of energy at different rates, provided on a monthly basis, thus requiring the scaling.
* mtr_val_old and mtr_val_new are meant to track the meter reading:
  * mtr_val_old will be prior to adding the invoice's usage, and mtr_val_new will be after.
  * The usage ("usage_n") is the sum of usage_1 -> usage_4, all multiplied by 'mtr_coef'.
  * This applies most of the time.
     * 'mtr_tariff' == 9 is a notable exception; its 'usage_n' is added to coinciding 'mtr_tariff' == 11 invoice.
* On 'mtr_coef':
  * If mtr_coef > 1, and [usage_3|usage_4] > 0, monthly cap seems to be 200.
  * If mtr_coef > 1, and [usage_3|usage_4] > 0, 'mtr_code' == [483|5__].

Conclusions based on the above:
* A current invoice's mtr_val_old should match previous invoice's mtr_val_new if there was no gap in between. It can be more if there is a gap. It should not be less.
* Similarly, the months_num should equal the difference in invoice dates in months (if there were no gaps). It can be less if there is a gap. It should not be more.
* The mtr_val_old plus the usage should equal mtr_val_new. Where a relevant 'mtr_tariff' == 9 is present, it too should be accounted for.
* The delta of the mtr_val_new of the current invoice and previous invoice should be energy used during the delta of the two invoice dates.

### Data Quality Markers

Going to use some helper functions that identify data inconsistencies as compared to expectations.
#### Intra-Row: Energy Usage
* 'usage_N' is sum of the 'usage_{1|2|3|4}' columns.
* 'usage_N' / 'mtr_coef' = 'usage_n' which represents, presumably, the amount being paid for.
* 'mtr_val_old' + 'usage_n' = 'mtr_val_new'.

'Check_Usage()' checks a series of rules based on expectations. 'Calc_Usage()' calculates these metrics and derives them via different possible permutations.

In [13]:
df_train['usage_flag'] = 0 # Does usage_n match delta of meter values? {0|1|-1} = {Y|N|N w 3 units rounding}
df_train['usage_n_aux_flag'] = 0 # Does usage_n_aux match delta of meter values? {0|1|-1} = {Y|N|N w 3 units rounding}

In [29]:
def Check_Usage(df_train):
    rows = len(df_train)
    # Doing checks:
    mask_1 = df_train['usage_n_calc'] != df_train['usage_n'] # Does usage match
    mask_2 = (df_train['usage_n_calc'] - df_train['usage_n']).abs() > 2 # Allow for rounding
    df_train.loc[mask_1, 'usage_flag'] = 1
    df_train.loc[mask_2, 'usage_flag'] = -1
    print(f"Rows with unexpected \'usage_n\' values: (allowing for rounding) not allowing / Total Rows: ({sum(mask_2)}) {sum(mask_1)} / {rows}.")
    mask_3 = (df_train['usage_aux_flag'] == 1)
    mask_4 = (df_train['usage_aux_flag'] == -1)
    print(f"Rows allowing \'usage_n_aux\' / not allowing: {sum(mask_3)} / {sum(mask_4)}. Combined: {sum(mask_3|mask_4)}.")
    mask_5 = (df_train['usage_n_calc'] != df_train['usage_n_aux']) # Does usage match, Already reverts to 'usage_n' if blocked
    mask_6 = ((df_train['usage_n_calc'] - df_train['usage_n_aux']).abs() > 2) # Allow for rounding
    df_train.loc[mask_5, 'usage_n_aux_flag'] = 1
    df_train.loc[mask_6, 'usage_n_aux_flag'] = -1
    print(f"Rows with unexpected \'usage_n_aux\' values: (allowing for rounding) not allowing: ({sum(mask_6)}) {sum(mask_5)}. Blocked \'usage_n_aux\': {sum(mask_4)}.")
    return df_train
def Calc_Usage(df_train):
    df_train['usage_N'] = df_train.loc[:, ['usage_1', 'usage_2', 'usage_3', 'usage_4']].sum(axis=1)
    df_train['usage_n'] = (df_train['usage_N'] / df_train['mtr_coef']).round(0)
    df_train['monthly_usage'] = df_train['usage_n'].div(df_train['months_num'])
    df_train['monthly_usage_aux'] = df_train['usage_aux'].div(df_train['months_num'])
    df_train['usage_n_aux'] = df_train['usage_n'] + (df_train['usage_aux']*((df_train['usage_aux_flag'] == 1).astype(int)))
    # Calculating derived metrics:
    df_train['usage_n_calc'] = df_train['mtr_val_new'] - df_train['mtr_val_old']
    df_train['usage_aux_calc'] = df_train['usage_n_calc'] - df_train['usage_n']
    df_train['mtr_coef_calc'] = (df_train['usage_N'] / df_train['usage_n_calc']).round(1)
    df_train['mtr_val_new_calc'] = df_train['mtr_val_old'] + df_train['usage_n'].round(0)
    df_train['mtr_val_old_calc'] = df_train['mtr_val_new'] - df_train['usage_n'].round(0)
    return df_train

In [30]:
df_train = Calc_Usage(df_train)
df_train = Check_Usage(df_train)

Rows with unexpected 'usage_n' values: (allowing for rounding) not allowing / Total Rows: (17339) 17533 / 4476749.
Rows allowing 'usage_n_aux' / not allowing: 0 / 0. Combined: 0.
Rows with unexpected 'usage_n_aux' values: (allowing for rounding) not allowing: (17339) 17533. Blocked 'usage_n_aux': 0.


#### Inter-Row: Meter and Date Consistency
* 'mtr_val_{old|new}' should monotonically increase over time. 
* 'invoice_date' - 'months_num' >= Prev('invoice_date') and should strictly increase over time.

'Calc_Order()' calculates relevant metrics that are then tested in 'Check_Order()'. Since 'months_num' is an integer and vague in its definition, will be adding 1 month leeway to calculations.

In [31]:
df_train['mtr_flag_bkd'] = 0 # Does mtr_val_old overlap mtr_val_new_prv? {0|1|-1} = {Y|N|N w < 3 units rounding}
df_train['mtr_flag_fwd'] = 0 # Does mtr_val_new overlap mtr_val_old_nxt? {0|1|-1} = {Y|N|N w < 3 units rounding}
df_train['date_flag_invoice'] = 0 # Does (invoice_date - months_num) overlap invoice_date_prv? {0|1|-1} = {Y|N|N w < 3 months rounding}
df_train['date_flag_months'] = 0 # Does months_num exceed (invoice_date - invoice_date_prv)? {0|1|-1} = {Y|N|N w < 3 months rounding}

In [46]:
def Check_Order(df_train):
    rows = len(df_train)
    # Doing checks:
    mask_1 = (df_train['mtr_val_new_prv'].notna())
    mask_2 = ((df_train['mtr_val_old'] < df_train['mtr_val_new_prv']) & mask_1)
    mask_3 = (((df_train['mtr_val_old'] - df_train['mtr_val_new_prv']).abs() > 2) & mask_1)
    df_train.loc[mask_1, 'mtr_flag_bkd'] = 1
    df_train.loc[mask_2, 'mtr_flag_bkd'] = -1
    print(f"Rows with backward overlapping meters: (allowing for rounding) not allowing / Total Rows: ({sum(mask_3)}) {sum(mask_2)} / {rows-sum(mask_1)}. Excluded NAs: {sum(mask_1)}.")
    mask_1 = (df_train['mtr_val_old_nxt'].notna())
    mask_2 = ((df_train['mtr_val_new'] > df_train['mtr_val_old_nxt']) & mask_1)
    mask_3 = (((df_train['mtr_val_new'] - df_train['mtr_val_old_nxt']).abs() > 2) & mask_1)
    df_train.loc[mask_1, 'mtr_flag_fwd'] = 1
    df_train.loc[mask_2, 'mtr_flag_fwd'] = -1
    print(f"Rows with forward overlapping meters: (allowing for rounding) not allowing / Total Rows: ({sum(mask_3)}) {sum(mask_2)} / {rows-sum(mask_1)}. Excluded NAs: {sum(mask_1)}.")
    mask_1 = (df_train['invoice_date_prv_calc'].notna())
    mask_2 = ((df_train['invoice_date_prv_calc'] < df_train['invoice_date_prv']) & mask_1)
    mask_3 = (((np.absolute((df_train['invoice_date_prv_calc'] - df_train['invoice_date_prv']).dt.days.values) / 30.5) > 2) & mask_1)
    df_train.loc[mask_1, 'date_flag_invoice'] = 1
    df_train.loc[mask_2, 'date_flag_invoice'] = -1
    print(f"Rows with overlapping invoice spans: (allowing for rounding) not allowing / Total Rows: ({sum(mask_3)}) {sum(mask_2)} / {rows-sum(mask_1)}. Excluded NAs: {sum(mask_1)}.")
    mask_2 = ((df_train['months_num_calc'] < df_train['months_num']) & mask_1)
    mask_3 = (((df_train['months_num_calc'] - df_train['months_num']).abs() > 2) & mask_1)
    df_train.loc[mask_1, 'date_flag_months'] = 1
    df_train.loc[mask_2, 'date_flag_months'] = -1   
    print(f"Rows with overlapping invoice spans: (allowing for rounding) not allowing / Total Rows: ({sum(mask_3)}) {sum(mask_2)} / {rows-sum(mask_1)}. Excluded NAs: {sum(mask_1)}.")
    return df_train
def Calc_Order(df_train, check, grp_cols=grp_cols):
    # Group, Find neighbours
    df_grp = df_train.groupby(grp_cols, observed=True)
    df_train['mtr_val_new_prv_2'] = df_grp['mtr_val_new'].shift(2)
    col_names = ['mtr_val_old', 'mtr_val_new', 'invoice_date', 'months_num', 'monthly_usage', 'monthly_usage_aux']
    df_train[[col_name+'_prv' for col_name in col_names]] = (df_grp[col_names].shift(1))
    df_train[[col_name+'_nxt' for col_name in col_names[:2]]] = (df_grp[col_names[:2]].shift(-1))
    df_train['mtr_val_old_nxt_2'] = df_grp['mtr_val_old'].shift(-2)
    # Calculate expected metrics based on relationships
    df_train['invoice_date_prv_calc'] = pd.NaT # If months_num is unreasonable it would crash
    mask = (df_train['months_num'] > 0) & (df_train['months_num'] < 600) & (df_train['months_num'].notna()) # 0 - 50 Years
    df_train.loc[mask, 'invoice_date_prv_calc'] = df_train.loc[mask, 'invoice_date'] - pd.to_timedelta(df_train.loc[mask, 'months_num'].astype(int) * 30.5, unit='days')
    df_train.loc[~mask, 'invoice_date_prv_calc'] = np.nan
    df_train['months_num_calc'] = (((df_train['invoice_date'] - df_train['invoice_date_prv']).dt.days.values / 30.5)).round(1)
    df_train['months_gap_calc'] = df_train['months_num_calc'] - df_train['months_num'] # Difference in dates in months
    if check: 
        df_train = Check_Order(df_train.copy())
    return df_train

In [47]:
df_train = Calc_Order(df_train, check=False)
df_train = Check_Order(df_train)

Rows with backward overlapping meters: (allowing for rounding) not allowing / Total Rows: (1589833) 597816 / 233780. Excluded NAs: 4242969.
Rows with forward overlapping meters: (allowing for rounding) not allowing / Total Rows: (1589833) 597816 / 233780. Excluded NAs: 4242969.
Rows with overlapping invoice spans: (allowing for rounding) not allowing / Total Rows: (1521200) 2221006 / 1406. Excluded NAs: 4475343.
Rows with overlapping invoice spans: (allowing for rounding) not allowing / Total Rows: (1510530) 2075308 / 1406. Excluded NAs: 4475343.


In [ ]:
def Calc_Neighbours(df_train, check, grp_cols=grp_cols):
    rows = len(df_train)
    # Group, Find neighbours
    df_grp = df_train.groupby(grp_cols, observed=True)
    df_train['mtr_val_new_prv_2'] = df_grp['mtr_val_new'].shift(2)
    col_names = ['mtr_val_old', 'mtr_val_new', 'invoice_date', 'months_num', 'monthly_usage', 'monthly_usage_aux']
    df_train[[col_name+'_prv' for col_name in col_names]] = (df_grp[col_names].shift(1))
    df_train[[col_name+'_nxt' for col_name in col_names[:2]]] = (df_grp[col_names[:2]].shift(-1))
    df_train['mtr_val_old_nxt_2'] = df_grp['mtr_val_old'].shift(-2)
    # Calculate expected metrics based on relationships
    df_train['invoice_date_prv_calc'] = pd.NaT # If months_num is unreasonable it would crash
    mask = (df_train['months_num'] > 0) & (df_train['months_num'] < 600) & (df_train['months_num'].notna()) # 0 - 50 Years
    df_train['invoice_date_prv_calc'] = df_train.loc[mask, 'invoice_date'] - pd.to_timedelta(df_train.loc[mask, 'months_num'].astype(int) * 30.5, unit='days')
    df_train['months_num_calc'] = np.ceil(((df_train['invoice_date'] - df_train['invoice_date_prv']).dt.days.values / 30.5))
    df_train['months_gap_calc'] = df_train['months_num_calc'] - df_train['months_num'] # Difference in dates in months
    # Check Rules
    mask = ((df_train['mtr_val_old'] < df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
    df_train['mtr_flag_bkd'] = mask.astype(int)
    print(f"Rows where meter readings seem out of order looking backwards / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
    mask = ((df_train['mtr_val_new'] > df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
    df_train['mtr_flag_fwd'] = mask.astype(int)
    print(f"Rows where meter readings seem out of order looking forwards / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['mtr_val_old_nxt'].isna())}.")
    mask = ((df_train['invoice_date_prv_calc'] + pd.to_timedelta(30.5, unit='days') < df_train['invoice_date_prv']) & (df_train['invoice_date_prv_calc'].notna()))
    df_train['date_flag'] = mask.astype(int)
    print(f"Rows where invoice spans seem to overlap based on \'months_num\' / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['invoice_date_prv'].isna())}.")
    mask = ((df_train['months_num_calc'] < df_train['months_num']) & (df_train['invoice_date_prv_calc'].notna()))
    df_train.loc[mask, 'date_flag'] = 1
    print(f"Rows where invoice spans seem to overlap based on \'invoice_date\'s / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['invoice_date_prv'].isna())}.")
    return df_train.copy()
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows where meter readings seem out of order looking backwards / Total Rows: 597816 / 4476749. Excluded NAs: 233780.
Rows where meter readings seem out of order looking forwards / Total Rows: 597816 / 4476749. Excluded NAs: 233780.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 1014829 / 4476749. Excluded NAs: 233780.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 1014871 / 4476749. Excluded NAs: 233780.


### Data Corruption

#### Column Misalignment
##### Focusing on 'mtr_status'
'mtr_status' had some suspiciously rare values that were manually inspected: [3, 2, 46, A, 628, 769, 269375, 420]. 

From these, Values: ['46', '618', '769', '269375', '420'] seemed symptomatic of data quality issues.

In [15]:
mask = df_train['mtr_status'].isin(['46', '618', '769', '269375', '420']) # Manual check showed 'A' seems acceptable
print(f'number of bad mtr_status: {sum(mask)}.')
col_names = ['mtr_tariff', 'mtr_id', 'mtr_status', 'mtr_code', 'mtr_notes', 'mtr_coef', 'usage_1', 'usage_2', 'usage_3', 'usage_4', 'mtr_val_old', 'mtr_val_new', 'months_num']
pd.concat([df_train[~mask].head(5), df_train[mask].head(5)])[col_names] # Comparison

number of bad mtr_status: 34.


,mtr_tariff,mtr_id,mtr_status,mtr_code,mtr_notes,mtr_coef,usage_1,usage_2,usage_3,usage_4,mtr_val_old,mtr_val_new,months_num
22,11,1335667,0,203,6,1,124,0,0,0,3685.0,3809.0,4
23,11,1335667,0,203,6,1,141,0,0,0,3809.0,3950.0,4
24,11,1335667,0,203,6,1,162,0,0,0,3950.0,4112.0,4
25,11,1335667,0,203,6,1,159,0,0,0,4112.0,4271.0,4
28,11,1335667,0,203,6,1,182,0,0,0,4271.0,4453.0,4
1178214,11,170,769,0,207,6,1,332,0,0,0.0,0.0,332
1178211,11,170,769,0,207,6,1,385,0,0,0.0,332.0,717
1178200,11,170,769,0,207,6,1,479,0,0,0.0,717.0,1196
1178209,11,170,769,0,207,6,1,437,0,0,0.0,1196.0,1633
1178207,11,170,769,0,207,6,1,453,0,0,0.0,1633.0,2086


For these, it is assumed that the columns have been shifted by mistake. One key sign here is usage_1 == 1 and mtr_coef == 6 | 8 | 9.

It is not clear why. In particular, it is not clear what 'mtr_id' and/or 'mtr_status' is meant to represent here and which might be wrong. 'mtr_id' will be kept and 'mtr_status' will be deleted.

'months_num' will be empty for now.

In [16]:
# Flag and modify
df_train['col_shift_flag'] = mask.astype(int)
df_train.loc[mask, df_train.columns[4:14]] = df_train.loc[mask, df_train.columns[5:15]].values
df_train.loc[mask, df_train.columns[14]] = np.nan
df_train = Calc_Usage(df_train.copy())
pd.concat([df_train[~mask].head(5), df_train[mask].head(5)])[col_names] # Comparison

Rows with unexpected usage values / Total Rows: 17502 / 4476749.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 0 / 17502.
Mismatched Rows if allowing for rounding errors / Total Rows: 4459393 / 4476749.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 0 / 0.


,mtr_tariff,mtr_id,mtr_status,mtr_code,mtr_notes,mtr_coef,usage_1,usage_2,usage_3,usage_4,mtr_val_old,mtr_val_new,months_num
22,11,1335667,0,203,6,1,124,0,0,0,3685.0,3809.0,4.0
23,11,1335667,0,203,6,1,141,0,0,0,3809.0,3950.0,4.0
24,11,1335667,0,203,6,1,162,0,0,0,3950.0,4112.0,4.0
25,11,1335667,0,203,6,1,159,0,0,0,4112.0,4271.0,4.0
28,11,1335667,0,203,6,1,182,0,0,0,4271.0,4453.0,4.0
1178214,11,170,0.0,207,6,1,332,0,0,0,0.0,332.0,NaN
1178211,11,170,0.0,207,6,1,385,0,0,0,332.0,717.0,NaN
1178200,11,170,0.0,207,6,1,479,0,0,0,717.0,1196.0,NaN
1178209,11,170,0.0,207,6,1,437,0,0,0,1196.0,1633.0,NaN
1178207,11,170,0.0,207,6,1,453,0,0,0,1633.0,2086.0,NaN


##### Focusing on 'mtr_coef'
One identified issue is assumed to be caused by 'mtr_ceof' having had a decimal place that caused an offset in columns. This was deduced by suspicious 'months_num' and 'usage_2' values originally.

For example, "1,5" would cause the "5" to fall into 'usage_1', etc. 'months_num' was then overwritten and lost.

'mtr_coef' would therefore need to be reconstructed for these examples, and columns shifted. 

The lost 'months_num' are empty for now.

Rows have to be filtered carefully to identify only those related to the described data corruption problem:
* If(0 < 'usage_1' < 10), then flag.
  * On the assumption it is single decimal place, it must be a single digit.
  * All other conditions are applied after this.
* If('usage_2' > 0) or If('months_num' > 240), then flag.
  * The minimum monthly cap seen is 50, so 'usage_2' should not be used prior to that.
  * This would not apply if only true 'usage_1' was used, so is insufficient.
  * 'months_num' would not reasonably exceed 20 years.

In [17]:
# Finding those affected
mask = (((df_train['usage_1'] > 0) & (df_train['usage_1'] < 10)) &
        ((df_train['usage_2'] != 0)) | (df_train['months_num'] > 240))
df_temp = df_train[mask]
col_names = ['mtr_coef', 'usage_1', 'usage_2', 'usage_3', 'usage_4', 'mtr_val_old', 'mtr_val_new', 'months_num', 'usage_N', 'usage_n', 'usage_n_calc']
pd.concat([df_train[~mask].head(5), df_train[mask].head(5)])[col_names] # Comparison

,mtr_coef,usage_1,usage_2,usage_3,usage_4,mtr_val_old,mtr_val_new,months_num,usage_N,usage_n,usage_n_calc
22,1,124,0,0,0,3685.0,3809.0,4.0,124,124.0,124.0
23,1,141,0,0,0,3809.0,3950.0,4.0,141,141.0,141.0
24,1,162,0,0,0,3950.0,4112.0,4.0,162,162.0,162.0
25,1,159,0,0,0,4112.0,4271.0,4.0,159,159.0,159.0
28,1,182,0,0,0,4271.0,4453.0,4.0,182,182.0,182.0
20214,1,5,1200,3024,0,0.0,495.0,3311.0,4229,4229.0,495.0
20219,1,5,229,0,0,0.0,342.0,495.0,234,234.0,342.0
20212,1,5,1200,10566,0,0.0,9971.0,17815.0,11771,11771.0,9971.0
20213,1,5,1200,8790,0,0.0,3311.0,9971.0,9995,9995.0,3311.0
20211,1,5,1200,10744,0,0.0,17815.0,25778.0,11949,11949.0,17815.0


In [18]:
# Enact the change, and keep record
df_train.loc[mask, 'col_shift_flag'] = 1
df_train['unk_months_num'] = mask.astype(int)
df_train['mtr_coef'] = df_train['mtr_coef'].astype(float)
df_train.loc[mask, 'mtr_coef'] = df_train.loc[mask, 'mtr_coef'] + (df_train.loc[mask, 'usage_1'].astype(float) / 10).round(1)
df_train.loc[mask, df_train.columns[8:14]] = df_train.loc[mask, df_train.columns[9:15]].values
df_train.loc[mask, df_train.columns[14]] = np.nan
print(f"Number of Rows where Columns seem offset based on ruleset / Total Rows: {sum(mask)} / {len(df_train)}.")
df_train = Calc_Usage(df_train.copy())
df_train[mask][col_names]

Number of Rows where Columns seem offset based on ruleset / Total Rows: 1392 / 4476749.
Rows with unexpected usage values / Total Rows: 16436 / 4476749.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 0 / 16436.
Mismatched Rows if allowing for rounding errors / Total Rows: 4460759 / 4476749.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 0 / 0.


,mtr_coef,usage_1,usage_2,usage_3,usage_4,mtr_val_old,mtr_val_new,months_num,usage_N,usage_n,usage_n_calc
20214,1.5,1200,3024,0,0,495.0,3311.0,NaN,4224,2816.0,2816.0
20219,1.5,229,0,0,0,342.0,495.0,NaN,229,153.0,153.0
20212,1.5,1200,10566,0,0,9971.0,17815.0,NaN,11766,7844.0,7844.0
20213,1.5,1200,8790,0,0,3311.0,9971.0,NaN,9990,6660.0,6660.0
20211,1.5,1200,10744,0,0,17815.0,25778.0,NaN,11944,7963.0,7963.0
...,...,...,...,...,...,...,...,...,...,...,...
4457223,1.5,200,100,200,1810,459733.0,461273.0,NaN,2310,1540.0,1540.0
4457217,1.5,1000,407,0,0,465008.0,465946.0,NaN,1407,938.0,938.0
4457218,1.5,200,100,200,239,463554.0,464047.0,NaN,739,493.0,493.0
4457226,1.5,200,100,200,1321,458519.0,459733.0,NaN,1821,1214.0,1214.0


#### Poor Date Parsing
Another identified issue is assumed caused by the original uploader of the dataset to Kaggle relying on automated date parsing, resulting in months and days being switched in places. This was deduced by considering the 'invoice_date', 'mtr_val_old', 'mtr_val_new', and 'months_num' fields. The significance is that 'months_num' is calculated on true 'invoice_date' and not the recorded 'invoice_date' here.

Using the data hierarchy mentioned before, it would broadly be expected for 'mtr_val_old' and 'mtr_val_new' to increase monotonically.

In [ ]:
# Finding those affected
mask = (df_train['invoice_date'].dt.day <= 12) & (df_train['invoice_date'].dt.month <= 12)
print(f"Rows where meter readings day and month could have been switched / Total Rows: {sum(mask)} / {len(df_train)}.") # ~1/3 of times
print(f"Rows where meter readings seem out of order / Rows where day and month could have been switched: {sum(((df_train['mtr_flag_bkd'] == 1) | (df_train['mtr_flag_fwd'] == 1)) & mask)} / {sum(mask)}.") # ~1/3 of times

Rows where meter readings day and month could have been switched / Total Rows: 1976635 / 4476749.
Rows where meter readings seem out of order / Rows where day and month could have been switched: 998921 / 1976635.


In [ ]:
# Flip the day and month for these cases, and keep record. There might be a vectorised way for this to speed this up...
df_train['date_flip_flag'] = mask.astype(int)
df_train.loc[mask, 'invoice_date'] = df_train.loc[mask, 'invoice_date'].apply(lambda x: x.replace(day=x.month, month=x.day) if pd.notna(x) else x)
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows where meter readings seem out of order looking backwards / Total Rows: 11881 / 4476749. Excluded NAs: 233780.
Rows where meter readings seem out of order looking forwards / Total Rows: 11881 / 4476749. Excluded NAs: 233780.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 4480 / 4476749. Excluded NAs: 233780.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 4525 / 4476749. Excluded NAs: 233780.


Given how significantly the data quality metric improved, it is unlikely to be due to chance. However, there is still a chance that some did not need switching, for which is nothing is done, unfortunately.

### Data Coherence

#### "Empty" Rows
Some rows are essentially empty. Either they have no recorded usage nor meter values, or similar. These mainly seem out of place, and do not provide much value. These will be removed unless it would not leave at least one row for each 'client_id' and 'mtr_type'.

'Remove_Rows()' function will handle this task.

In [21]:
def Remove_Rows(df_train, mask, reason, df_removed=None):
    df_train['temp_flag'] = mask
    # Must have at least one record kept for the group
    mask = mask & (df_train.groupby(['client_id', 'mtr_type'], observed=True)['temp_flag'].transform('min') == 0)
    df_temp = df_train[mask].copy()
    df_temp['removed'] = reason
    if df_removed is not None:
        df_removed = pd.concat([df_removed, df_temp.copy()], sort=False)
    else:
        df_removed = df_temp.copy()
    print(f'Rows to be removed / out of rows requested to be removed: {len(df_temp)} / {sum(df_train['temp_flag'])}.')
    df_train = df_train[~mask]
    return (df_train.copy(), df_removed.copy(), df_temp)

##### "Empty" 'mtr_coef':
'mtr_coef' of 0 is rare, and any energy usage in that row does not transfer onwards. These primarily are all equally rare 'mtr_tariff' of 8. It was checked that these were unrelated previous 'mtr_coef' issue, i.e., they were not 0.6 etc.

In [22]:
mask = df_train['mtr_coef'] == 0 # Manual check showed these do not contribute to energy usage over time. They reset each row.
df_train['bad_coef'] = mask.astype(int)
df_train, df_removed, df_temp = Remove_Rows(df_train, mask, 'mtr_empty')
df_temp[col_names].head()

Rows to be removed / out of rows requested to be removed: 45 / 45.


,mtr_coef,usage_1,usage_2,usage_3,usage_4,mtr_val_old,mtr_val_new,months_num,usage_N,usage_n,usage_n_calc
179746,0.0,0,0,0,0,0.0,87.0,4.0,0,NaN,87.0
179751,0.0,0,0,0,0,0.0,0.0,4.0,0,NaN,0.0
179755,0.0,0,0,0,0,0.0,633.0,4.0,0,NaN,633.0
997145,0.0,0,0,0,0,0.0,0.0,1.0,0,NaN,0.0
1228208,0.0,0,0,0,0,0.0,0.0,4.0,0,NaN,0.0


##### "Empty" Meter: Focusing on ['usage_{1|2|3|4}', 'mtr_val_{old|new}']
There are some rows where 'n_usage' == 'mtr_val_old' == 'mtr_val_new' == 0. These will be removed as their overall data quality is low, and those that do not seem out of place, simply provide little value. Note that entire 'mtr_id' groups will not be removed.

In [ ]:
mask = (df_train[['usage_N', 'mtr_val_old', 'mtr_val_new']] == 0).all(axis=1) # These are considered empty
df_train, df_removed, df_temp = Remove_Rows(df_train, mask, 'mtr_empty', df_removed)
df_temp[col_names].head()
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows to be removed / out of rows requested to be removed: 103849 / 195401.
Rows with unexpected usage values / Total Rows: 16391 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 0 / 16391.
Mismatched Rows if allowing for rounding errors / Total Rows: 4356910 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 0 / 0.
Rows where meter readings seem out of order looking backwards / Total Rows: 11333 / 4372855. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11333 / 4372855. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3720 / 4372855. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3749 / 4372855. Excluded NAs: 228999.


##### No Activity: Focusing on ['usage_{1|2|3|4}', 'mtr_val_{old|new}']
There are also some rows where 'n_usage' == 0 where 'mtr_val_old' == 'mtr_val_new' != 0. These will not be removed, but will be treated with lower priority if in conflict with another row. This is done by the being sorted by 'usage_n' such that, if all else equal, they'd appear before a row with 'usage_n' > 0, and so, will be enveloped by it.

#### Intra-Row: Mis-matched Usage
##### "Stuck" Meter:
There seems to be an issue with the 'mtr_val_{old|new}' not reflecting 'usage_n'. The cause is not clear, assumed some sort of stuck meter.
These will be changed if one of two checks are met:
  1. recalculating 'mtr_val_new' using 'mtr_val_old' and 'usage_n' aligns with the next row. 
  2. recalculating 'mtr_val_old' using 'mtr_val_new' and 'usage_n' aligns with the previous row. 

A rounding error of < 3 units will be allowed for. Done sequentially. This is not ideal as some cases will be missed. A lot of these cases will be removed later downstream.

In [ ]:
# Checking with next row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = ((df_train['mtr_val_old_nxt'] - df_train['mtr_val_new_calc']).abs() > 2) | (df_train['mtr_val_old_nxt'].isna()) # Don't trust, even allow for rounding error
df_train.loc[mask & mask_2, 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
print(f"Looking at Next Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Next Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Next Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Checking with previous row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = ((df_train['mtr_val_new_prv'] - df_train['mtr_val_old_calc']).abs() > 2) | (df_train['mtr_val_old_nxt'].isna()) # Don't trust, even allow for rounding error
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
df_train.loc[(df_train['mtr_stuck'] != 1) & (mask & mask_2), 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
print(f"Looking at Previous Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Previous Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Previous Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Re-sort and check for the same rule again
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)
df_temp = df_train[df_train['mtr_stuck'] == -1]
col_names = ['mtr_coef', 'usage_N', 'usage_n', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'months_num', 'mtr_val_new_calc']
df_temp[col_names].head()

Looking at Next Row: Number of Rows detected: 7281.
Looking at Next Row: Number of Rows deemed unfixable: 4403.
Looking at Next Row: Number of Rows deemed fixable: 2878.
Looking at Previous Row: Number of Rows detected: 4403.
Looking at Previous Row: Number of Rows deemed unfixable: 3876.
Looking at Previous Row: Number of Rows deemed fixable: 527.
Rows with unexpected usage values / Total Rows: 12986 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 0 / 12986.
Mismatched Rows if allowing for rounding errors / Total Rows: 4360220 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 0 / 0.
Rows where meter readings seem out of order looking backwards / Total Rows: 11642 / 4372855. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11642 / 4372855. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3720 / 437

,mtr_coef,usage_N,usage_n,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num,mtr_val_new_calc
1865,1.0,2943,2943.0,7709.0,7709.0,7709.0,7709.0,8.0,10652.0
2865,1.0,5000,5000.0,3143.0,3143.0,3143.0,3143.0,18.0,8143.0
3004,1.0,204,204.0,1469.0,1469.0,1469.0,1469.0,12.0,1673.0
6864,1.0,20,20.0,157.0,157.0,157.0,157.0,4.0,177.0
6918,1.0,44,44.0,15678.0,15702.0,15702.0,15810.0,2.0,15746.0


##### "Phantom" Usage
An almost reverse of the above problem is when there is no apparent usage (usage_N == 0), but 'mtr_val_old' != 'mtr_val_new'.

One identified cause was an interaction with 'mtr_tariff' == 9 where its usage would be added to coinciding 'mtr_tariff' == 11 invoice's meter values. However, this seems quite inconsistant. I am likely not understanding the full picture here, so will try to demonstrate logic used.

Exploring instances of a given 'invoice_date' containing multiple 'mtr_tariff' values, that included 'mtr_tariff' == 9, for a given 'client_id' and 'mtr_id'. From these, looking at how many times the 'usage_n' for given 'mtr_tariff' was non-zero, it seems clear only 'mtr_tariff' == 11 is relevant here.

In [25]:
mask = (df_train['mtr_tariff'] == 9)
df_train['temp_flag'] = mask
df_train['temp_flag'] = df_train.groupby(['client_id', 'mtr_id'], observed=True)['temp_flag'].transform('max') # Client / Meters with tariff 9
mask = (df_train['mtr_val_old'] != df_train['mtr_val_new']) & (df_train['usage_N'] == 0) # those with the missing usage
df_train['temp_flag'] = mask
df_train['temp_flag'] = df_train.groupby(['client_id', 'mtr_id'], observed=True)['temp_flag'].transform('max')
# Make Copy and keep only relevant rows
df_temp = df_train.copy()
df_temp.sort_values(by=['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff'], inplace=True) # Resort 
df_temp = df_temp[df_temp['temp_flag'] == True]
col_names = df_temp.loc[df_temp['temp_flag'] == True, 'mtr_tariff'].unique().tolist()
col_names.sort()
# Pivot and get sum of usage_n vs usage_n_calc for each mtr_tariff for each invoice_date + mtr_id + client_id
df_temp = df_temp.groupby(['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff'], observed=True)[['usage_n', 'usage_n_calc']].sum().unstack('mtr_tariff', fill_value=-999999999).reset_index()
# Remove rows that have only one mtr_tariff active:
mask = (df_temp[list(zip(['usage_n']*len(col_names), col_names))] != -999999999).sum(axis=1) > 1
df_temp = df_temp[mask]
# Go through each of these mtr_tariff (except 9), and remove those where all instances match as they are not relevant
mask = []
for col_name in col_names:
        df_temp[('match', col_name)] = df_temp[('usage_n', col_name)] == df_temp[('usage_n_calc', col_name)]
        if (col_name != 9) & (sum(df_temp[('match', col_name)]) == len(df_temp)):
            df_temp.drop(columns=[('usage_n', col_name), ('usage_n_calc', col_name), ('match', col_name)], inplace=True)
            mask = mask + [col_name]
col_names = list(filter(lambda x: x not in mask, col_names))
print(f"Number of non-zero rows for \'mtr_tariff\' values / total rows: {col_names}: {[sum(df_temp[('usage_n_calc', col_name)] != 0) for col_name in col_names]} / {len(df_temp)}")
# Know that only columns 9 and 11 matter after running this once
df_temp.drop(columns=[('usage_n', 10), ('usage_n_calc', 10), ('match', 10)], inplace=True)
# Checking whether combining usage from mtr_tariff == 9 would actually help:
df_temp[('usage_n', '11_adj')] = df_temp[('usage_n', 9)] + df_temp[('usage_n', 11)]
df_temp[('match', '11_adj')] = df_temp[('usage_n', '11_adj')] == df_temp[('usage_n_calc', 11)]
df_temp[('delta', 11)] = df_temp[('usage_n_calc', 11)] - df_temp[('usage_n', 11)]
df_temp[('delta', '11_adj')] = df_temp[('usage_n_calc', 11)] - df_temp[('usage_n', '11_adj')]
mask = df_temp[('match', '11_adj')] == False
mask_2 = df_temp[('match', 11)] == False
print(f"Rows \'mtr_tariff\' == 11 usage seemed unexpected, rows when adding \'mtr_tariff\' == 9 usage / rows when not adding: {sum(mask)} / {sum(mask_2)}. (Lower is better!)")
mask = df_temp[('delta', '11_adj')].abs() < df_temp[('delta', 11)].abs()
print(f"Rows where the change reduced the delta in unexpected usage, rows when adding \'mtr_tariff\' == 9 usage / total rows: {sum(mask)} / {len(df_temp)}. (Higher is better!)")

Number of non-zero rows for 'mtr_tariff' values / total rows: [9, 10, 11]: [3969, 4154, 4067] / 4159
Rows 'mtr_tariff' == 11 usage seemed unexpected, rows when adding 'mtr_tariff' == 9 usage / rows when not adding: 2363 / 3918. (Lower is better!)
Rows where the change reduced the delta in unexpected usage, rows when adding 'mtr_tariff' == 9 usage / total rows: 3320 / 4159. (Higher is better!)


Over 75% where fixed and over 95% were at least improved so it is clearly not just a coincidence. The process is repeated below, but with all instances of mtr_tariff == 9|11 included, not just when there is unexpected usage, to ensure nothing is missed.

In [26]:
mask = (df_train['mtr_tariff'] == 9) | (df_train['mtr_tariff'] == 11) # Only care about these two
df_train['temp_flag'] = mask
# Make Copy and keep only relevant rows
df_temp = df_train.copy()
df_temp.sort_values(by=['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff'], inplace=True) # Resort 
df_temp = df_temp[df_temp['temp_flag'] == True]
col_names = df_temp.loc[df_temp['temp_flag'] == True, 'mtr_tariff'].unique().tolist()
col_names.sort()
# Pivot and get sum of usage_n vs usage_n_calc for each mtr_tariff for each invoice_date + mtr_id + client_id
df_temp = df_temp.groupby(['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff'], observed=True)[['usage_n', 'usage_n_calc']].sum().unstack('mtr_tariff', fill_value=-999999999).reset_index()
# Remove rows that have only one mtr_tariff active:
mask = (df_temp[list(zip(['usage_n']*len(col_names), col_names))] != -999999999).sum(axis=1) > 1
df_temp = df_temp[mask]
# Check which instances match
for col_name in col_names:
        df_temp[('match', col_name)] = df_temp[('usage_n', col_name)] == df_temp[('usage_n_calc', col_name)]
print(f"Number of non-zero rows for \'mtr_tariff\' values / total rows: {col_names}: {[sum(df_temp[('usage_n_calc', col_name)] != 0) for col_name in col_names]} / {len(df_temp)}")
df_temp[('usage_n', '11_adj')] = df_temp[('usage_n', 9)] + df_temp[('usage_n', 11)]
df_temp[('match', '11_adj')] = df_temp[('usage_n', '11_adj')] == df_temp[('usage_n_calc', 11)]
df_temp[('delta', 11)] = df_temp[('usage_n_calc', 11)] - df_temp[('usage_n', 11)]
df_temp[('delta', '11_adj')] = df_temp[('usage_n_calc', 11)] - df_temp[('usage_n', '11_adj')]
mask = df_temp[('match', '11_adj')] == False
mask_2 = df_temp[('match', 11)] == False
print(f"Rows \'mtr_tariff\' == 11 usage seemed unexpected, rows when adding \'mtr_tariff\' == 9 usage / rows when not adding: {sum(mask)} / {sum(mask_2)}. (Lower is better!)")
mask = df_temp[('delta', '11_adj')].abs() < df_temp[('delta', 11)].abs()
print(f"Rows where the change reduced the delta in unexpected usage, rows when adding \'mtr_tariff\' == 9 usage / total rows: {sum(mask)} / {len(df_temp)}. (Higher is better!)")

Number of non-zero rows for 'mtr_tariff' values / total rows: [9, 11]: [5334, 5509] / 5644
Rows 'mtr_tariff' == 11 usage seemed unexpected, rows when adding 'mtr_tariff' == 9 usage / rows when not adding: 2394 / 5297. (Lower is better!)
Rows where the change reduced the delta in unexpected usage, rows when adding 'mtr_tariff' == 9 usage / total rows: 4678 / 5644. (Higher is better!)


Therefore, this change was adopted and these values were placed into the previously created 'usage_aux' column.

In [ ]:
df_temp['usage_n_9'] = df_temp[('usage_n', 9)] # Can never get my head around this multi level stuff :\
df_temp['mtr_tariff'] = 11
df_temp.drop(columns=[('usage_n', 9), ('usage_n', 11),
                       ('usage_n_calc', 9), ('usage_n_calc', 11), 
                      ('match', 9), ('match', 11), 
                      ('usage_n', '11_adj'), ('match', '11_adj'), 
                      ('delta', 11), ('delta', '11_adj')], inplace=True)
df_temp.columns = df_temp.columns.get_level_values(0)
df_train = df_train.merge(df_temp[['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff', 'usage_n_9']], on=['client_id', 'mtr_id', 'invoice_date', 'mtr_tariff'], how='left')
mask = df_train['usage_n_9'].notna()
df_train.loc[mask, 'usage_aux_flag'] = 1
df_train.loc[mask, 'usage_aux'] = df_train.loc[mask, 'usage_n_9']
df_train.drop(columns=['usage_n_9'], inplace=True)
# Revert to original sort and check usage once more
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Usage(df_train.copy())

Rows with unexpected usage values / Total Rows: 12986 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2978 / 10008.
Mismatched Rows if allowing for rounding errors / Total Rows: 4360220 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 3251 / 11.


Going to reject those cases where 'usage_aux' was not helpful:
* If(abs(delta(using 'usage_aux')) < abs(delta(!using 'usage_aux'))) and If(delta(using 'usage_aux') > -10), then accept
   * As long as it does not 'overshoot' by too much, can be accepting of almost any improvement.
* If(abs(delta(using 'usage_aux')) x 4 < abs(delta(!using 'usage_aux'))), then accept
   * Otherwise, would want a substantial improvement to justify inclusion.

In [ ]:
mask = df_train['usage_aux_flag'] == 1 # Those that had overlapping mtr_tariff == 9
df_train['temp_flag'] = np.nan
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'usage_n_calc'] - df_train.loc[mask, 'usage_n']
mask_2 = (df_train['temp_flag'] - df_train['usage_aux']) # Residual delta after making usage of 'usage_aux'
mask_2 = (((mask_2.abs() * 4) < (df_train['temp_flag']).abs()) |
          ((mask_2.abs() < df_train['temp_flag'].abs()) & (mask_2 > -10)))
print(f"Rows where \'usage_aux\' is rejected / rows it was present: {sum(mask & ~mask_2)} / {sum(mask)}.")
df_train.loc[(mask & ~mask_2), 'usage_aux_flag'] = -1 # These are flagged as rejected, although they are retained for now
df_train = Calc_Usage(df_train.copy())
df_temp = df_train[(mask & ~mask_2)]
df_temp

Rows where 'usage_aux' is rejected / rows it was present: 1499 / 5645.
Rows with unexpected usage values / Total Rows: 12986 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2978 / 10008.
Mismatched Rows if allowing for rounding errors / Total Rows: 4360220 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 2978 / 10.


,client_id,invoice_date,mtr_tariff,mtr_id,mtr_status,mtr_code,mtr_notes,mtr_coef,usage_1,usage_2,...,months_num_calc,months_gap_calc,mtr_flag_bkd,mtr_flag_fwd,date_flag,col_shift_flag,unk_months_num,date_flip_flag,bad_coef,mtr_stuck
8437,train_Client_100244,2014-05-15,11,630735,0,413,8,1.0,400,200,...,NaN,NaN,0,0,0,0,0,0,0,NaN
17318,train_Client_100479,2017-02-14,11,21557018982,0,207,9,1.0,0,0,...,NaN,NaN,0,0,0,0,0,0,0,NaN
17319,train_Client_100479,2017-06-09,11,21557018982,0,207,9,1.0,0,0,...,4.0,0.0,0,0,0,0,0,1,0,NaN
17321,train_Client_100479,2018-02-13,11,21557018982,0,207,9,1.0,0,0,...,4.0,2.0,0,0,0,0,0,0,0,NaN
17322,train_Client_100479,2018-06-06,11,21557018982,0,207,9,1.0,0,0,...,4.0,0.0,0,0,0,0,0,1,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366035,train_Client_99816,2015-10-20,11,100099,0,413,9,1.0,400,200,...,NaN,NaN,0,0,0,0,0,0,0,NaN
4366040,train_Client_99816,2017-06-13,11,100099,0,413,9,1.0,0,0,...,4.0,0.0,0,0,0,0,0,0,0,NaN
4366043,train_Client_99816,2018-06-11,11,100099,0,413,9,1.0,0,0,...,4.0,0.0,0,0,0,0,0,1,0,NaN
4366046,train_Client_99816,2019-06-21,11,100099,0,413,9,1.0,0,0,...,4.0,0.0,0,0,0,0,0,0,0,NaN


Recalculating the "Stuck Meter" using the 'usage_n_aux':

In [ ]:
# Checking with next row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = ((df_train['mtr_val_old_nxt'] - (df_train['mtr_val_old'] + df_train['usage_n_aux'])).abs() > 2) | (df_train['mtr_val_old_nxt'].isna()) # Don't trust, even allow for rounding error
df_train.loc[mask & mask_2, 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
print(f"Looking at Next Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Next Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Next Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Checking with previous row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = ((df_train['mtr_val_new_prv'] - (df_train['mtr_val_new'] - df_train['usage_n_aux'])).abs() > 2) | (df_train['mtr_val_new_prv'].isna()) # Don't trust, even allow for rounding error
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
df_train.loc[(df_train['mtr_stuck'] != 1) & (mask & mask_2), 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
print(f"Looking at Previous Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Previous Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Previous Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Re-sort and check for the same rule again
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)
df_temp = df_train[df_train['mtr_stuck'] == -1]
col_names = ['mtr_coef', 'usage_N', 'usage_n', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'months_num', 'mtr_val_new_calc']
df_temp[col_names].head()

Looking at Next Row: Number of Rows detected: 3876.
Looking at Next Row: Number of Rows deemed unfixable: 2629.
Looking at Next Row: Number of Rows deemed fixable: 1247.
Looking at Previous Row: Number of Rows detected: 2629.
Looking at Previous Row: Number of Rows deemed unfixable: 769.
Looking at Previous Row: Number of Rows deemed fixable: 1860.
Rows with unexpected usage values / Total Rows: 9879 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2978 / 6901.
Mismatched Rows if allowing for rounding errors / Total Rows: 4363322 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 2978 / 10.
Rows where meter readings seem out of order looking backwards / Total Rows: 13084 / 4372855. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 13084 / 4372855. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3720 

,mtr_coef,usage_N,usage_n,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num,mtr_val_new_calc
6765,1.0,44,44.0,15678.0,15702.0,15702.0,15810.0,2.0,15746.0
8893,1.0,81,81.0,21504.0,21683.0,21683.0,22161.0,2.0,21764.0
15562,1.0,187,187.0,20204.0,20492.0,20492.0,21370.0,2.0,20679.0
16194,1.0,25,25.0,5.0,62.0,62.0,183.0,2.0,87.0
25217,1.0,130,130.0,NaN,3406.0,3406.0,NaN,2.0,3536.0


If these rows are in the expected order in terms of dates and meter values (looking forwards), these instances are fixed. This done very simply by placing it all into 'usage_1'. Otherwise, these are left alone for now. Allowing for small rounding errors of < 3 units.

Often, it is the entire history of that 'mtr_tariff' & 'mtr_id' that has the issue. But sometimes, it is sporadic.

In [ ]:
mask = ((df_train['mtr_val_old'] != df_train['mtr_val_new']) & # Changing meter reading
        (((df_train['usage_N'] == 0) & (df_train['usage_aux_flag'] != 1)) | # Those with no usage and flagged to not use usage_aux
         ((df_train['usage_N'] + df_train['usage_aux'] == 0) & (df_train['usage_aux_flag'] != -1)))) # those with no usage flagged to use usage_aux
mask_2 = ((df_train['date_flag'] == 0) & ((df_train['mtr_val_old_nxt'] - df_train['mtr_val_new']).abs() < 3))
df_train['usage_missing'] = (mask & mask_2).astype(int) # Those being adjusted
df_train.loc[(mask & ~mask_2), 'usage_missing'] = -1 # Not being adjusted
df_train.loc[(mask & mask_2), 'usage_1'] = df_train.loc[(mask & mask_2), 'usage_n_calc'] * df_train.loc[(mask & mask_2), 'mtr_coef'] # Want unscaled value
print(f"Rows where meter changes whilst no usage is recorded, those adjusted / All possibly affected : {sum(mask & mask_2)} / {sum(mask)}.")
df_train = Calc_Usage(df_train.copy())
df_temp = df_train[(mask & ~mask_2)]
col_names = ['mtr_coef', 'usage_N', 'usage_n', 'usage_n_calc', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'months_num']
df_temp[col_names].head()

Rows where meter changes whilst no usage is recorded, those adjusted / All possibly affected : 28 / 1338.
Rows with unexpected usage values / Total Rows: 9851 / 4372855.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2978 / 6873.
Mismatched Rows if allowing for rounding errors / Total Rows: 4363343 / 4372855.
Mismatched Rows if allowing for rounding error, fixable via 'usage_aux' / those not fixed: 2978 / 10.


,mtr_coef,usage_N,usage_n,usage_n_calc,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num
8436,1.0,0,0.0,624.0,7110.0,7110.0,7734.0,NaN,2.0
17318,1.0,0,0.0,267.0,NaN,0.0,267.0,267.0,4.0
17319,1.0,0,0.0,202.0,267.0,267.0,469.0,469.0,4.0
17321,1.0,0,0.0,84.0,2069.0,2247.0,2331.0,2331.0,2.0
17322,1.0,0,0.0,158.0,2331.0,2331.0,2489.0,2489.0,4.0


#### Inter-Row: Mis-matched Usage
##### Meter "Re-Write" (1 of 2):
It seems that sometimes, the meter is "reset" to retroactively adjust values. A complication is that this somewhat overlaps with a seperate issue of meter "roll over" when it exceeds maximum number of digits.

The signs of this happening are:
1. 'invoice_date' - 'months_num' >= Prev('invoice_date')
2. 'mtr_val_old' >= 'mtr_val_new_prv' (alternatively, 'mtr_val_new' <= 'mtr_val_old_nxt')

Not all of those breaking the first rule seem out of order, however, neither does removing the row introduce an issue. The 'months_num' seems quite coarse so have to allow for rounding values.

It will typically be the row prior to the one that is "reset" that were "re-written" and will be removed to restore order.

This is split into two steps, in an attempt to disentangle it from the meter "roll over" issue. The logic used is quite repetitive, this is so different cases could be treated differently in the future if wanted. Otherwise, could be consolidated.

The general logic is in the 'Calc_Overlap()' function. Where a value (date or meter reading) relative to the prior row seems out of place, it is first flagged. Any row prior to this, that is out of order of said row, is then flagged for removal. If a row would be out of order compared to multiple rows, precedence is given to the most onerous case (i.e., the earliest date or lowest meter reading).

In [31]:
def Calc_Overlap(df_train, mask, col_name):
    df_train['temp_flag'] = df_train.groupby(grp_cols, observed=True)['temp_flag'].transform('shift', periods=-1) # Flag row before the newly overlapping row
    # Sometimes, one flagged instance is subsumed by another, using cumulative min in reverse order to override these. 
    df_temp = df_train[grp_cols + ['temp_flag']].dropna(subset=['temp_flag']) # Broken off into temp df to speed up as most as NaT values.
    df_temp['temp_flag'] = df_temp.groupby(grp_cols, observed=True)['temp_flag'].apply(lambda x: x[::-1].cummin()[::-1]).to_list() # Keep earliest value mentioned so far, per group, in reverse order
    df_train.loc[df_temp.index, 'temp_flag'] = df_temp['temp_flag'] # Bring it back in
    df_train['temp_flag'] = df_train.groupby(grp_cols, observed=True)['temp_flag'].bfill() # Flag all previous within the group
    df_train['temp_flag'] = (df_train[col_name] > df_train['temp_flag']) & (df_train['temp_flag'].notna()) # Flag only those within the overlapping period
    mask = df_train['temp_flag'] == True
    return df_train, mask

Looking at those breaking both rules.

In [32]:
mask = (df_train['date_flag'] == 1) # overlapping months
mask_2 = (df_train['mtr_flag_bkd'] == 1) | (df_train['mtr_flag_fwd'] == 1) # overlapping meters
print(f"Rows where meter readings also seem to be reset / Rows that seem out of order: {sum(mask & mask_2)} / {sum(mask)}.")
mask = mask & mask_2
df_train['temp_flag'] = pd.to_datetime(np.nan)
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'invoice_date_prv_calc'] + pd.to_timedelta(30.5, unit='days') # This is the date being sought after
df_train, mask = Calc_Overlap(df_train, mask, 'invoice_date') # Flag rows within overlapping period of subsequent rows
print(f"Rows affected by reset meter readings / total Rows: {sum(mask)} / {len(df_train)}.")
df_temp = df_train[mask]
col_names = ['invoice_date_prv', 'invoice_date_prv_calc', 'months_num', 'invoice_date', 'mtr_val_old_prv', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'mtr_val_new_nxt']
df_temp[col_names].head()

Rows where meter readings also seem to be reset / Rows that seem out of order: 301 / 3749.
Rows affected by reset meter readings / total Rows: 430 / 4372855.


,invoice_date_prv,invoice_date_prv_calc,months_num,invoice_date,mtr_val_old_prv,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,mtr_val_new_nxt
9146,2007-12-17,2007-12-06,2.0,2008-02-05,0.0,23475.0,23475.0,23475.0,22348.0,23475.0
22545,2010-12-14,2011-02-08,4.0,2011-06-10,3377.0,3745.0,3745.0,3745.0,0.0,4471.0
26308,2007-09-26,2007-12-09,2.0,2008-02-08,11395.0,11397.0,11771.0,11771.0,11397.0,11771.0
55720,2009-01-05,2009-08-29,2.0,2009-10-29,6666.0,6666.0,6666.0,6666.0,6609.0,6609.0
60096,2014-06-25,2014-06-27,8.0,2015-02-26,6313.0,6743.0,6743.0,43089.0,6743.0,6743.0


In [33]:
# Remove those flagged
df_train['mtr_reset'] = mask.astype(int)
df_train, df_removed, df_temp = Remove_Rows(df_train.copy(), mask, 'mtr_reset', df_removed.copy())
df_temp[col_names].head()
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows to be removed / out of rows requested to be removed: 430 / 430.
Rows where meter readings seem out of order looking backwards / Total Rows: 11105 / 4372425. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11105 / 4372425. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3423 / 4372425. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3452 / 4372425. Excluded NAs: 228999.


Looking at those breaking rule 1 but not rule 2:
* If meter currently in perfect alignment:
    * Only remove prior rows if also leads to perfect meter alignment ('mtr_val_old' == 'mtr_val_new_prv')
* If meter currently in order but not in perfect alignment:
    * Only remove prior rows if leads to meter being in order ('mtr_val_old' >= 'mtr_val_new_prv')

The logic here is to not worsen the state of alignment by the changes.

In [34]:
# Looking at those currently in perfect meter alignment
mask = (df_train['date_flag'] == 1) & (((df_train['mtr_val_new'] == df_train['mtr_val_old_nxt']) | (df_train['mtr_val_old_nxt'].notna())) |
                                       ((df_train['mtr_val_old'] == df_train['mtr_val_new_prv']) | (df_train['mtr_val_new_prv'].notna())))
df_train['temp_flag'] = pd.to_datetime(np.nan)
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'invoice_date_prv_calc'] + pd.to_timedelta(30.5, unit='days') # This is the date being sought after
df_train, mask = Calc_Overlap(df_train, mask, 'invoice_date') # Flag rows within overlapping period of subsequent rows
# Want to check whether removing these improves alignment.
df_temp = df_train[df_train['temp_flag'] == False].copy() # Comparison case
print("vvv Ignore outputs below! vvv")
df_temp = Calc_Sort(df_temp.copy(), 'usage_n')
df_temp = Calc_Neighbours(df_temp.copy(), grp_cols) # Update values
print("^^^ Ignore outputs above! ^^^")
df_temp['temp_flag_2'] = df_temp['mtr_val_old'] != df_temp['mtr_val_new_prv'] # Not perfect alignment
df_train['temp_flag_2'] = df_train['mtr_val_old'] != df_train['mtr_val_new_prv'] 
# Only those that reduces number of rows that have non-perfect alignment kept
mask = (df_temp.groupby(grp_cols, observed=True)['temp_flag_2'].agg('sum') < df_train.groupby(grp_cols, observed=True)['temp_flag_2'].agg('sum')).reset_index()
mask = mask[mask['temp_flag_2'] == True]
df_train['temp_flag_2'] = df_train.set_index(grp_cols).index.isin(mask.set_index(grp_cols).index) # Flagging the groups where it was improvement
mask = (df_train['temp_flag'] == True) & (df_train['temp_flag_2'] == True)
print(f'Rows with overlapping periods that would otherwise align if removed / Total overlapping rows: {sum(mask)} / {sum(df_train['temp_flag'] == 1)}.')

vvv Ignore outputs below! vvv
Rows where meter readings seem out of order looking backwards / Total Rows: 11067 / 4368964. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11067 / 4368964. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 0 / 4368964. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 30 / 4368964. Excluded NAs: 228999.
^^^ Ignore outputs above! ^^^
Rows with overlapping periods that would otherwise align if removed / Total overlapping rows: 166 / 3461.


In [35]:
# Remove those flagged
df_train.loc[mask, 'mtr_reset'] = 1
df_train, df_removed, df_temp = Remove_Rows(df_train.copy(), mask, 'mtr_reset', df_removed.copy())
df_temp[col_names].head()
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows to be removed / out of rows requested to be removed: 166 / 166.
Rows where meter readings seem out of order looking backwards / Total Rows: 11070 / 4372259. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11070 / 4372259. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3272 / 4372259. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3301 / 4372259. Excluded NAs: 228999.


In [36]:
# Looking at those currently in order but not in perfect meter alignment
mask =  (df_train['date_flag'] == 1) & (((df_train['mtr_val_new'] < df_train['mtr_val_old_nxt']) | (df_train['mtr_val_old_nxt'].notna())) |
                                       ((df_train['mtr_val_old'] > df_train['mtr_val_new_prv']) | (df_train['mtr_val_new_prv'].notna())))
df_train['temp_flag'] = pd.to_datetime(np.nan)
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'invoice_date_prv_calc'] + pd.to_timedelta(30.5, unit='days') # This is the date being sought after
df_train, mask = Calc_Overlap(df_train, mask, 'invoice_date') # Flag rows within overlapping period of subsequent rows
# Want to check whether removing these causes good alignment.
df_temp = df_train[df_train['temp_flag'] == False].copy() # Comparison case
print("vvv Ignore outputs below! vvv")
df_temp = Calc_Sort(df_temp.copy(), 'usage_n')
df_temp = Calc_Neighbours(df_temp.copy(), grp_cols) # Update values
print("^^^ Ignore outputs above! ^^^")
df_temp['temp_flag_2'] = df_temp['mtr_val_old'] < df_temp['mtr_val_new_prv']  # Not in order (previous test was for perfect alignment)
df_train['temp_flag_2'] = df_train['mtr_val_old'] < df_train['mtr_val_new_prv'] 
# Only those that reduces number of misaligned rows kept
mask = (df_temp.groupby(grp_cols, observed=True)['temp_flag_2'].agg('sum') < df_train.groupby(grp_cols, observed=True)['temp_flag_2'].agg('sum')).reset_index()
mask = mask[mask['temp_flag_2'] == True]
df_train['temp_flag_2'] = df_train.set_index(grp_cols).index.isin(mask.set_index(grp_cols).index) # Flagging the groups where it was improvement
mask = (df_train['temp_flag'] == True) & (df_train['temp_flag_2'] == True)
print(f'Rows with overlapping periods that stay aligned if removed / Total overlapping rows: {sum(mask)} / {sum(df_train['temp_flag'] == 1)}.')

vvv Ignore outputs below! vvv
Rows where meter readings seem out of order looking backwards / Total Rows: 11067 / 4368964. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11067 / 4368964. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 0 / 4368964. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 30 / 4368964. Excluded NAs: 228999.
^^^ Ignore outputs above! ^^^
Rows with overlapping periods that stay aligned if removed / Total overlapping rows: 3 / 3295.


In [37]:
# Remove those flagged
df_train.loc[mask, 'mtr_reset'] = 1
df_train, df_removed, df_temp = Remove_Rows(df_train.copy(), mask, 'mtr_reset', df_removed.copy())
df_temp[col_names].head()
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows to be removed / out of rows requested to be removed: 3 / 3.
Rows where meter readings seem out of order looking backwards / Total Rows: 11067 / 4372256. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 11067 / 4372256. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3269 / 4372256. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3298 / 4372256. Excluded NAs: 228999.


Note that this leaves behind two sets of cases:
1. Those with overlapping invoice spans but with meters in order: where removing rows degrades alignment. 
   * Sub-set of breaking rule 1 but not rule 2.
2. Those that do not have overlapping 'months_num' but with meters out of order. 
   * Breaking rule 2 but not rule 1.

Will return to these later.

#### "Reset" Meter: Rollover
There is also a "rollover" issue with the 'mtr_val_{old|new}' columns, where some reset after exceeding 100000 | 1000000 back down to 0. Two scenarios:
* Rollover occurs within a row (intra-row)
* Rollover occurs between a row (it is assumed this requires a "gap" in dates) (inter-row)

One complication is that it is not clear what the cap is for a given meter, and this has to be inferred based on what caused the rollover. Where possible, looking to add amount to fix issue.

##### Inter-Row:
Unfortunately, it seems quite hard to distinguish meter rolling over from a meter re-write at times. The settled upon rules are:
* Sufficient Time:
   * Must have unaccounted gap in invoice spans in which meter could have rolled over.
* Sufficient Usage:
   * Previous month's monthly usage projected forwards into the gap must be enough to increase number of digits.
   * OR, Current month's monthly usage project backwards into the gap must be enough to cause negative values.
* Qualifiers:
   * The alleged rollover number of digits must be greater than seen prior in the group:
       * For example, cannot claim a rollover at 10000 if there exists a meter readings >10000 already.
       * It must also be greater than assumed minimum: 9999.
   * The relied upon monthly usage cannot be too abnormal:
       * Within (Median/3) < usage < (Median*3), ignoring those with 0 usage.
   * Meter must not be locked at 0 for both mtr_val_old and mtr_val_new.

In [39]:
# Store max digits seen for a group, assumed constant and that rollover could not happen less than this.
df_train['mtr_max_digit'] = df_train[['mtr_val_old', 'mtr_val_new']].max(axis=1) # value per row
df_train['mtr_max_digit'] = df_train.groupby(grp_cols, observed=True)[['mtr_max_digit']].transform('max') # value per group
df_train['mtr_max_digit'] = 10 ** np.ceil(np.log10(df_train['mtr_max_digit'] + 1)) # Number of digits
# Looking at rollover across rows
df_train['monthly_usage_grp_median'] = df_train['monthly_usage'].replace(0, np.nan)
df_train['monthly_usage_grp_median'] = df_train.groupby(grp_cols, observed=True)['monthly_usage_grp_median'].transform('median')
mask = ((df_train['months_gap_calc'] > 1) & # Must have a gap
        (df_train['usage_flag'] == 0) & # Usage must make sense
        (df_train['monthly_usage'] < (df_train['monthly_usage_grp_median'] * 3)) & # Usage not more than triple median 
        ((df_train['monthly_usage'] * 3) > df_train['monthly_usage_grp_median'])) # Usage not less than third median
# Looking if current usage would have caused rollover going backwards:
df_train['temp_flag'] = np.nan
df_train.loc[mask, 'temp_flag'] = (df_train.loc[mask, 'monthly_usage'] * df_train.loc[mask, 'months_gap_calc']).round(0) # Projected usage
mask_2 = ((df_train['mtr_val_old'] < df_train['temp_flag']) & # Had enough (gap * usage) to have caused rollover
          (df_train['mtr_val_new_prv'] + df_train['temp_flag'] > df_train['mtr_max_digit']) & # Would have caused highest digit count seen
          (df_train['mtr_val_new_prv'] + df_train['temp_flag'] > 9999) & # Minimum rollover point assumed
          (((df_train['mtr_val_new_prv'] + df_train['temp_flag']) - (df_train['mtr_val_old'] + df_train['mtr_max_digit'])).abs() < 2000))
# Looking if previous usage would have caused rollover going forwards:
df_train['temp_flag'] = np.nan
df_train.loc[mask, 'temp_flag'] = (df_train.loc[mask, 'monthly_usage_prv'] * df_train.loc[mask, 'months_gap_calc']).round(0) # Projected usage
mask = ((df_train['mtr_val_new_prv'] + df_train['temp_flag'] > df_train['mtr_max_digit']) & # Would have caused highest digit count seen (and so also rollover)
        (df_train['mtr_val_new_prv'] + df_train['temp_flag'] > 9999) & # Minimum rollover point assumed
        (((df_train['mtr_val_new_prv'] + df_train['temp_flag']) - (df_train['mtr_val_old'] + df_train['mtr_max_digit'])).abs() < 2000))
print(f"Rows suspected of inter-row rollover projecting backwards / those projecting forwards: {sum(mask_2)} / {sum(mask)}.")
# Ignore those that have mtr_val_old and mtr_val_new == 0
mask = (mask | mask_2) 
mask_2 = (df_train['mtr_val_old'] == 0) & (df_train['mtr_val_new'] == 0) 
print(f"Rows suspected of inter-row rollover, deemed fixable / Rows suspected: {sum(mask & ~mask_2)} / {sum(mask)}.")
df_train['mtr_old_rollover'] = (mask & ~mask_2).astype(int)
df_train.loc[(mask & mask_2), 'mtr_old_rollover'] = -1
col_names = ['client_id', 'invoice_date', 'usage_n', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'months_num', 'mtr_val_new_calc', 'mtr_max_digit']
df_temp = df_train[(mask & mask_2) if sum(mask & mask_2)>0 else (mask & ~mask_2)] # Showing those unfixable if they exist, otherwise, just those flagged
df_temp[col_names]

Rows suspected of inter-row rollover projecting backwards / those projecting forwards: 92 / 152.
Rows suspected of inter-row rollover, deemed fixable / Rows suspected: 185 / 185.


,client_id,invoice_date,usage_n,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num,mtr_val_new_calc,mtr_max_digit
1622,train_Client_100044,2012-12-11,492.0,9977.0,219.0,711.0,711.0,4.0,711.0,10000.0
8876,train_Client_100257,2010-01-26,2171.0,8992.0,2860.0,5031.0,7420.0,4.0,5031.0,10000.0
25205,train_Client_100717,2011-10-13,556.0,9537.0,1429.0,1985.0,1985.0,4.0,1985.0,10000.0
128944,train_Client_103569,2012-12-17,2971.0,85269.0,1305.0,4276.0,NaN,4.0,4276.0,100000.0
131353,train_Client_103636,2012-04-13,127.0,8863.0,3802.0,3929.0,4116.0,2.0,3929.0,10000.0
...,...,...,...,...,...,...,...,...,...,...
4178558,train_Client_94567,2016-04-22,12682.0,997907.0,7745.0,20427.0,20427.0,6.0,20427.0,1000000.0
4206476,train_Client_95331,2015-05-04,220.0,6501.0,2925.0,3145.0,3145.0,4.0,3145.0,10000.0
4298613,train_Client_97944,2009-08-04,4371.0,97197.0,1478.0,5849.0,5849.0,1.0,5849.0,100000.0
4304767,train_Client_98106,2014-05-26,704.0,9794.0,2066.0,2770.0,2770.0,2.0,2770.0,10000.0


##### Intra-Row:
Rules for these:
* Expected values must exceed maximum number of digits seen in a group, and exceed 9999
* EITHER: Monthly usage cannot be too abnormal; within: (Median/3) < usage < (Median*3), ignoring those with 0 usage.
* OR: meter values must align, absolute delta < 3.

In [40]:
mask = ((df_train['monthly_usage'] < (df_train['monthly_usage_grp_median'] * 3)) & # Usage not more than triple median 
        ((df_train['monthly_usage'] * 3) > df_train['monthly_usage_grp_median'])) # Usage not less than third median
mask_2 = (((df_train['mtr_val_new_calc'] - (df_train['mtr_val_new'] + (df_train['usage_n'] + (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1))))).abs() < 3) &
          (df_train['mtr_val_old_nxt'].notna())) # Usage and Meters would align, allowing for rounding errors
mask = (mask | mask_2)
mask_2 = ((df_train['mtr_val_new_calc'] > df_train['mtr_max_digit']) & 
          (df_train['mtr_val_new_calc'] > 9999)) # number of digits > most seen, and > Minimum rollover point
mask = (mask & mask_2)
# From these, only going to make the change if they don't cause alignment issue to their next row after change, and those without blank meter values
mask_2 = ((((df_train['mtr_val_new_calc'] - df_train['mtr_max_digit']) - 2 <= df_train[ 'mtr_val_old_nxt']) | # -2 is to allow for rounding errors
          (df_train['mtr_val_old_nxt'].isna())) &
          ((df_train['mtr_val_old'] != 0) | (df_train['mtr_val_new'] != 0)))

print(f"Rows suspected of intra-row rollover, those deemed fixable / Rows suspected: {sum(mask & mask_2)} / {sum(mask)}.")
df_train['mtr_new_rollover'] = mask.astype(int)
df_train.loc[(mask & (~mask_2)), 'mtr_new_rollover'] = -1 
df_temp = df_train[(mask & (~mask_2)) if sum(mask & (~mask_2))>0 else (mask & mask_2)] # Showing those unfixable if they exist, otherwise, just those flagged
df_temp[col_names]

Rows suspected of intra-row rollover, those deemed fixable / Rows suspected: 1977 / 1981.


,client_id,invoice_date,usage_n,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num,mtr_val_new_calc,mtr_max_digit
1997989,train_Client_33768,2011-11-16,14111.0,NaN,0.0,0.0,NaN,4.0,14111.0,1.0
2674970,train_Client_52765,2009-05-15,18287.0,1674.0,1674.0,1674.0,1674.0,32.0,19961.0,10000.0
3365437,train_Client_72015,2007-11-08,18117.0,NaN,0.0,0.0,NaN,4.0,18117.0,1.0
3635271,train_Client_79521,2007-01-23,20043.0,2226.0,2226.0,2226.0,2226.0,16.0,22269.0,10000.0


Once a rollover occurs, it has a knock-on effects. One issue is that multiple rollovers can occur. 
* Inter-Row: 'mtr_old_rollover':
  * 'mtr_val_old' and 'mtr_val_new' needs adjustment every instance.
* Intra-Row: 'mtr_new_rollover':
  * The first instance it occurs, only 'mtr_val_new' needs adjusting, but then all rows after need both that and 'mtr_val_old' adjusting.

The expected 'mtr_val_new_calc' is recalculated after this once more.

In [41]:
# Keeping a track of change made to mtr_val_old and mtr_val new. Combining both mtr_old_rollover and mtr_new_rollover before making the change.
df_train[['mtr_old_adj_quant', 'mtr_new_adj_quant']] = 0
col_names = ['temp_flag', 'temp_flag_2']
for mask_name in ['mtr_old_rollover', 'mtr_new_rollover']:
    mask = df_train[mask_name] == 1
    df_train[col_names] = np.nan
    df_train.loc[mask, col_names] = df_train.loc[mask,'mtr_max_digit']
    df_train.loc[mask, col_names] = df_train[mask].groupby(grp_cols, observed=True)[col_names].cumsum() # Count number of instances
    df_train[col_names] = df_train.groupby(grp_cols, observed=True)[col_names].ffill()
    df_train[col_names] = df_train[col_names].fillna(0)
    df_train['mtr_old_adj_quant'] = df_train['mtr_old_adj_quant'] + df_train['temp_flag']
    df_train['mtr_new_adj_quant'] = df_train['mtr_new_adj_quant'] + df_train['temp_flag_2']
    print(f"Rows updated due to {mask_name}: {sum(df_train['temp_flag_2'] > 0)}.")
mask = df_train['mtr_new_rollover'] == 1
df_train.loc[mask,'mtr_old_adj_quant'] = df_train.loc[mask,'mtr_old_adj_quant'] - df_train.loc[mask,'mtr_max_digit'] # Ignore first instance for new_rollover
# Update the values
df_train['mtr_val_old'] = df_train['mtr_val_old'] + df_train['mtr_old_adj_quant']
df_train['mtr_val_new'] = df_train['mtr_val_new'] + df_train['mtr_new_adj_quant']

Rows updated due to mtr_old_rollover: 1796.
Rows updated due to mtr_new_rollover: 26257.


In [42]:
# Re-sort and check for the same rule again
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows with unexpected usage values / Total Rows: 10392 / 4372256.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2977 / 7415.
Rows where meter readings seem out of order looking backwards / Total Rows: 10900 / 4372256. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 10900 / 4372256. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3269 / 4372256. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3298 / 4372256. Excluded NAs: 228999.


#### Inter-Row: Mis-matched Usage (Continued)
##### Meter "Re-Write" (2 of 2):
Returning to the two sets of cases left previously:
1. Those with overlapping invoice spans but with meters in order: where removing rows degrades alignment. 
   * Sub-set of breaking rule 1 but not rule 2.
2. Those that do not have overlapping 'months_num' but with meters out of order. 
   * Breaking rule 2 but not rule 1.

This section will be a lot more coercive due to time constraints - need to get them in order, even if sacrificing some of the data to do so. Based on this exploration, it seems clear that no column with be ideal to use. 'mtr_val_new' was deemed the overall best metric to rely upon as it should be aligned directly with 'invoice_date', which is assumed always correct.

Looking at case 2: those without overlapping 'months_num' but with meters out of order. Could look to do something about those that have a more "hard" reset, i.e. has consistent history, and then starts again from 0 for another consistent history. For example, these could be split into new different pseudo-'mtr_id' entities. However, given how rare it is and time constraints, this was not done for simplicity here. Similarly, some of these may have been modified to have only the violating data imputed versus being deleted outright.

In [45]:
mask = ((df_train['mtr_flag_bkd'] != 0)) & (df_train['date_flag'] == 0)
df_train['temp_flag'] = np.nan
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'mtr_val_old'] # This is value to stay under
df_train, mask = Calc_Overlap(df_train, mask, 'mtr_val_new') # Flag rows within overlapping period of subsequent rows
print(f'Rows with out of order meter values / Total affected overlapping rows: {sum(((df_train['mtr_flag_bkd'] != 0)) & (df_train['date_flag'] == 0))} / {sum(mask)}.')

Rows with out of order meter values / Total affected overlapping rows: 10896 / 17816.


In [46]:
# Remove those flagged
df_train.loc[mask, 'mtr_reset'] = 1
df_train, df_removed, df_temp = Remove_Rows(df_train.copy(), mask, 'mtr_reset', df_removed.copy())
df_temp[col_names].head()
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows to be removed / out of rows requested to be removed: 17816 / 17816.
Rows where meter readings seem out of order looking backwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3262 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3289 / 4354440. Excluded NAs: 228999.


#### Imputing 'months_num'
There were some fields that were missing 'months_num' field due to prior data corruption. The importance of 'months_num' is that it gives a direct time-scale for 'usage_n'. This is especially important for the first record of a given group as there would be little-to-no other indications for time-scale otherwise. Now that the records are assumed broadly ordered, 'months_num' will be imputed. Ideally, the records would have had no 'gaps' over overlaps in 'invoice_date' periods, and so that could have been used to estimate 'months_num'. Unfortunately, that was shown to be quite unreliable. Instead, looking at other trends identified to impute where possible.

'monthly_cap' refers to the maximum value for 'usage_1' in cases where it is capped. The cap value for 'usage_1' linearly scales based on 'months_num', and so knowing the 'monthly_cap' and 'usage_1' would be enough to deduce 'months_num'. However, 'monthly_cap' is not a recorded field and must itself be estimated, it is nevertheless used here where possible.

An important note is that only 'mtr_tariff' == 10 | 11 have 'monthly_cap' values. The following rules were deduced by inspecting the data:
1. If('mtr_coef' > 1) and If(usage_1 == {200|300}), then ('monthly_cap' = {200|300})
2. If('mtr_coef' > 1) and If(usage_3 > 0), then ('monthly_cap' = 200)

In [47]:
df_train['monthly_cap'] = np.nan
mask = df_train['usage_2'] > 0 # only mtr_tariff 10 or 11 would apply to this
df_train.loc[mask, 'monthly_cap'] = df_train.loc[mask, 'usage_1'].div(df_train.loc[mask, 'months_num'])
print(f"Rows with missing \'months_num\': {sum(df_train['months_num'].isna())}.")
df_train['monthly_cap'].value_counts()

Rows with missing 'months_num': 1419.


monthly_cap
200.000000    333533
300.000000    191930
50.000000     126438
100.000000       450
133.333333       122
600.000000       110
150.000000        63
75.000000         40
450.000000        28
66.666667         15
400.000000         8
375.000000         6
360.000000         4
181.818182         4
62.500000          4
120.000000         3
60.000000          2
57.142857          2
900.000000         2
58.333333          2
166.666667         2
175.000000         2
55.555556          1
20.000000          1
Name: count, dtype: int64

In [48]:
# Apply Rule 1
mask = (df_train['months_num'].isna()) & ((df_train['usage_1'] == 200) | (df_train['usage_1'] == 300))
df_temp = df_train[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'monthly_cap'] = df_train.loc[mask, 'usage_1']
df_train.loc[mask, 'months_num'] = 1

Rows with 'months_num' adjusted / Rows missing 'months_num': (319/1419).


In [49]:
# Apply Rule 2
mask = (df_train['months_num'].isna()) & (df_train['usage_3'] > 0)
df_temp = df_train[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train['months_num_adj'] = 0
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'monthly_cap'] = 200
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'usage_1'] / 200

Rows with 'months_num' adjusted / Rows missing 'months_num': (27/1100).


From this point onwards, it is less clear how to proceed. Going to apply the following rules:
1. If('usage_2' > 0) and If('months_num_calc'/'usage_1' == {200|300}), then ('monthly_cap' = {200|300})
   * 'monthly_cap' = {200|300} are by far the most common values, so will accept them if they seem to fit.
2. If('usage_2' > 0) and If(('months_num_calc'-1)/'usage_1' == {200|300}), then ('monthly_cap' = {200|300})
   * 'months_num_calc' is not very reliable, and when it does not match 'months_num' it tends to be over by 1, so trying the variant to check.
3. If('usage_2' > 0) and If(('months_num_calc'+1)/'usage_1' == {200|300}), then ('monthly_cap' = {200|300})
   * Done after the previous to account for this being a rarer scenario.

In [53]:
# Apply Rule 1
mask = ((df_train['months_num'].isna()) & (df_train['usage_2'] > 0))
df_train['temp_flag'] = 0.0
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'usage_1'].div(df_train.loc[mask, 'months_num_calc'])
mask = mask & ((df_train['temp_flag'] == 200) | (df_train['temp_flag'] == 300))
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'monthly_cap'] = df_train.loc[mask, 'temp_flag']
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'usage_1'].div(df_train['monthly_cap'])

Rows with 'months_num' adjusted / Rows missing 'months_num': (43/1073).


In [54]:
# Apply Rule 2
mask = ((df_train['months_num'].isna()) & (df_train['usage_2'] > 0))
df_train['temp_flag'] = 0.0
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'usage_1'].div(df_train.loc[mask, 'months_num_calc']-1)
mask = mask & ((df_train['temp_flag'] == 200) | (df_train['temp_flag'] == 300))
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'monthly_cap'] = df_train.loc[mask, 'temp_flag']
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'usage_1'].div(df_train['monthly_cap'])

Rows with 'months_num' adjusted / Rows missing 'months_num': (24/1030).


In [55]:
# Apply Rule 3
mask = ((df_train['months_num'].isna()) & (df_train['usage_2'] > 0))
df_train['temp_flag'] = 0.0
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'usage_1'].div(df_train.loc[mask, 'months_num_calc']+1)
mask = mask & ((df_train['temp_flag'] == 200) | (df_train['temp_flag'] == 300))
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'monthly_cap'] = df_train.loc[mask, 'temp_flag']
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'usage_1'].div(df_train['monthly_cap'])

Rows with 'months_num' adjusted / Rows missing 'months_num': (0/1006).


For these final ones, many alternatives seem equally viable / dissatisfactory. Applying following rules:
1. If('mtr_tariff' == 10|11), then Backfill('monthly_cap'), then Frontfill('monthly_cap')
2. If('months_num' == na), and If('monthly_cap' != na), then ('months_num' = 'usage_1'/'monthly_cap')
3. If('months_num' == na), and If('monthly_num_calc' != na), then ('months_num' = 'months_num_calc')
4. If('months_num' == na), then ('months_num' = 'usage_n'/'monthly_usage_grp_median')
5. Else, ('months_num' = 4) <- The most common value.

In [56]:
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows with unexpected usage values / Total Rows: 10268 / 4354440.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2938 / 7330.
Rows where meter readings seem out of order looking backwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3262 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3289 / 4354440. Excluded NAs: 228999.


In [57]:
# Apply Rule 1
df_temp = df_train.groupby(grp_cols, observed=True)
df_train['monthly_cap'] = df_temp['monthly_cap'].bfill()
df_train['monthly_cap'] = df_temp['monthly_cap'].ffill()

In [58]:
# Apply Rule 2
mask = (df_train['months_num'].isna()) & (df_train['monthly_cap'].notna())
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'usage_1'].div(df_train.loc[mask, 'monthly_cap'])

Rows with 'months_num' adjusted / Rows missing 'months_num': (41/1006).


In [59]:
# Apply Rule 3
mask = (df_train['months_num'].isna()) & (df_train['months_num_calc'].notna())
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'months_num'] = df_train.loc[mask, 'months_num_calc']

Rows with 'months_num' adjusted / Rows missing 'months_num': (933/965).


In [60]:
# Apply Rule 4 
df_train['monthly_usage_grp_median'] = df_train['monthly_usage'].replace(0, np.nan)
df_train['monthly_usage_grp_median'] = df_train.groupby(grp_cols, observed=True)['monthly_usage_grp_median'].transform('median')
mask = (df_train['months_num'].isna()) & (df_train['monthly_usage_grp_median'].notna())
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'months_num'] = np.ceil(df_train.loc[mask, 'usage_n'].div(df_train.loc[mask, 'monthly_usage_grp_median']))

Rows with 'months_num' adjusted / Rows missing 'months_num': (3/32).


In [61]:
# Apply Rule 5
mask = (df_train['months_num'].isna())
df_temp = df_train.loc[mask]
print(f"Rows with \'months_num\' adjusted / Rows missing \'months_num\': ({sum(mask)}/{sum(df_train['months_num'].isna())}).")
df_train.loc[mask, 'months_num_adj'] = 1
df_train.loc[mask, 'months_num'] = 4

Rows with 'months_num' adjusted / Rows missing 'months_num': (29/29).


In [62]:
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Neighbours(df_train.copy(), grp_cols)

Rows with unexpected usage values / Total Rows: 10268 / 4354440.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2938 / 7330.
Rows where meter readings seem out of order looking backwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 2 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3264 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3296 / 4354440. Excluded NAs: 228999.


### Data Quality Check
Looking at the data once again after these changes to see where it stands.
* Rule 1a: 'mtr_val_old' + 'usage_n' + 'usage_aux' == 'mtr_val_new' (Only use 'usage_aux' if flagged)
   * Checking Usage matches Meter Values.
* Rule 1b: abs(('mtr_val_old' + 'usage_n' + 'usage_aux') - 'mtr_val_new') < 3 (Laxer Rule)
   * Checking Usage matches Meter Values (allows rounding errors).
* Rule 2a: 'mtr_val_new' == 'mtr_val_old_nxt'
   * Checking Meter Values Consistent between rows, looking forwards (expects no gap)
* Rule 2b: 'mtr_val_new' >= 'mtr_val_old_nxt' (Laxer Rule)
   * Checking Meter Values Consistent between rows, looking forwards (allows gap, expects no overlap)
* Rule 3a: 'mtr_val_old' == 'mtr_val_new_prv'
   * Checking Meter Values Consistent between rows, looking backwards (expects no gap)
* Rule 3b: 'mtr_val_old' <= 'mtr_val_new_prv' (Laxer Rule)
   * Checking Meter Values Consistent between rows, looking backwards (allows gap, expects no overlap)
* Rule 4a: 'months_num' == 'months_num_calc'
   * Checking Invoiced Dates Consistent between rows, looking backwards (expects no gap)
* Rule 4b: 'months_num' - 'months_num_calc' <= 1 (Laxer Rule)
   * Checking Invoiced Dates Consistent between rows, looking backwards (allows gap, expects <2M overlap)

These will indicate how well the data follows expected patterns. Failing the 'a' variant of rules and passing the 'b' variant may indicate missing rows whereas failing the 'b' variant indicates a potential problem. 

In [66]:
df_train['R_1a'] = (df_train['mtr_val_old'] + df_train['usage_n'] + (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1))) != df_train['mtr_val_new']
df_train['R_1b'] = ((df_train['mtr_val_old'] + df_train['usage_n'] + (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1))) - df_train['mtr_val_new']).abs() > 2
df_train['R_2a'] = ((df_train['mtr_val_new'] != df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
df_train['R_2b'] = ((df_train['mtr_val_new'] > df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
df_train['R_3a'] = ((df_train['mtr_val_old'] != df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_3b'] = ((df_train['mtr_val_old'] < df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_4a'] = ((df_train['months_num'] != df_train['months_num_calc']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_4b'] = (((df_train['months_num'] - df_train['months_num_calc']) > 1) & (df_train['mtr_val_new_prv'].notna()))

print(f"Rows breaking Rule 1a: {sum(df_train['R_1a'])} ({round(100 * sum(df_train['R_1a']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2a: {sum(df_train['R_2a'])} ({round(100 * sum(df_train['R_2a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_old_nxt'].isna())}.")
print(f"Rows breaking Rule 3a: {sum(df_train['R_3a'])} ({round(100 * sum(df_train['R_3a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 4a: {sum(df_train['R_4a'])} ({round(100 * sum(df_train['R_4a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
mask = (df_train['R_1a']) | (df_train['R_2a']) | (df_train['R_2b']) | (df_train['R_3a']) | (df_train['R_3b']) | (df_train['R_4a']) | (df_train['R_4b'])
print(f"Rows breaking any Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")

print(f"Rows breaking Rule 1b: {sum(df_train['R_1b'])} ({round(100 * sum(df_train['R_1b']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2b: {sum(df_train['R_2b'])} ({round(100 * sum(df_train['R_2b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 3b: {sum(df_train['R_3b'])} ({round(100 * sum(df_train['R_3b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 4b: {sum(df_train['R_4b'])} ({round(100 * sum(df_train['R_4b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
mask = (df_train['R_1b']) | (df_train['R_2b']) | (df_train['R_3b']) | (df_train['R_4b'])
print(f"Rows breaking any lax Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")

print(f"Rows removed: {len(df_removed)} ({round(len(df_removed) / (len(df_removed) + len(df_train)), 2)}%).")

Rows breaking Rule 1a: 7330 (0.2%).
Rows breaking Rule 2a: 285962 (6.6%). Excluded NAs: 228999.
Rows breaking Rule 3a: 285962 (6.6%). Excluded NAs: 228999.
Rows breaking Rule 4a: 1879509 (43.2%). Excluded NAs: 228999.
Rows breaking any Rule: 2027522 (46.6%).
Rows breaking Rule 1b: 6937 (0.2%).
Rows breaking Rule 2b: 2 (0.0%). Excluded NAs: 228999.
Rows breaking Rule 3b: 2 (0.0%). Excluded NAs: 228999.
Rows breaking Rule 4b: 2957 (0.1%). Excluded NAs: 228999.
Rows breaking any lax Rule: 9865 (0.2%).
Rows removed: 122309 (0.03%).


There are too many that break Rule 1. Over half were due to the "stuck" meter. Going to attempt to correct once more whilst being slightly more lax. As long as meter alignment is not broken, allowing for < 3 rounding error, they adjustment will be made.

In [79]:
# Checking with next row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = (((df_train['mtr_val_old_nxt'] - (df_train['mtr_val_new_calc'] + (df_train['usage_aux']*((df_train['usage_aux_flag'] == 1).astype(int)))) > 2)) |
          (df_train['mtr_val_old_nxt'].isna())) # Don't trust adjustment
df_train.loc[mask & mask_2, 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
print(f"Looking at Next Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Next Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Next Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Checking with previous row:
mask = ((df_train['mtr_val_old'] == df_train['mtr_val_new']) & (df_train['usage_n'] > 0)) # Meter not changing despite usage
mask_2 = (((df_train['mtr_val_new_prv'] - (df_train['mtr_val_old_calc'] - (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1).astype(int))) > 2)) |
          (df_train['mtr_val_new_prv'].isna())) # Don't trust adjustment
df_train.loc[mask & ~mask_2, 'mtr_stuck'] = 1 # Those being adjusted
df_train.loc[mask & ~mask_2, 'mtr_val_new'] = df_train.loc[mask & ~mask_2, 'mtr_val_new_calc'] # Replacing
df_train.loc[(df_train['mtr_stuck'] != 1) & (mask & mask_2), 'mtr_stuck'] = -1 # Those that wouldn't be fixed via adjustment
print(f"Looking at Previous Row: Number of Rows detected: {sum(mask)}.")
print(f"Looking at Previous Row: Number of Rows deemed unfixable: {sum(mask & mask_2)}.")
print(f"Looking at Previous Row: Number of Rows deemed fixable: {sum(mask & ~mask_2)}.")

# Re-sort and check for the same rule again
df_train = Calc_Usage(df_train.copy())
df_train = Calc_Sort(df_train.copy(), 'usage_n')
df_train = Calc_Neighbours(df_train.copy(), grp_cols)
df_temp = df_train[df_train['mtr_stuck'] == -1]
col_names = ['mtr_coef', 'usage_N', 'usage_n', 'mtr_val_new_prv', 'mtr_val_old', 'mtr_val_new', 'mtr_val_old_nxt', 'months_num', 'mtr_val_new_calc']
df_temp[col_names].head()

Looking at Next Row: Number of Rows detected: 2129.
Looking at Next Row: Number of Rows deemed unfixable: 2128.
Looking at Next Row: Number of Rows deemed fixable: 1.
Looking at Previous Row: Number of Rows detected: 2128.
Looking at Previous Row: Number of Rows deemed unfixable: 2128.
Looking at Previous Row: Number of Rows deemed fixable: 0.
Rows with unexpected usage values / Total Rows: 7846 / 4354440.
Rows with unexpected usage values, fixable via 'usage_aux' / those not fixed: 2938 / 4908.
Rows where meter readings seem out of order looking backwards / Total Rows: 1774 / 4354440. Excluded NAs: 228999.
Rows where meter readings seem out of order looking forwards / Total Rows: 1774 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'months_num' / Total Rows: 3264 / 4354440. Excluded NAs: 228999.
Rows where invoice spans seem to overlap based on 'invoice_date's / Total Rows: 3296 / 4354440. Excluded NAs: 228999.


,mtr_coef,usage_N,usage_n,mtr_val_new_prv,mtr_val_old,mtr_val_new,mtr_val_old_nxt,months_num,mtr_val_new_calc
6765,1.0,44,44.0,15678.0,15702.0,15702.0,15810.0,2.0,15746.0
7096,1.0,677,677.0,1705.0,1705.0,1705.0,2888.0,4.0,2382.0
12796,1.0,3297,3297.0,4651.0,4651.0,4651.0,8190.0,12.0,7948.0
20737,1.0,91,91.0,400.0,400.0,400.0,497.0,4.0,491.0
20781,1.0,313,313.0,72.0,72.0,72.0,460.0,4.0,385.0


In [71]:
df_train['R_1a'] = (df_train['mtr_val_old'] + df_train['usage_n'] + (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1))) != df_train['mtr_val_new']
df_train['R_1b'] = ((df_train['mtr_val_old'] + df_train['usage_n'] + (df_train['usage_aux']*(df_train['usage_aux_flag'] == 1))) - df_train['mtr_val_new']).abs() > 2
df_train['R_2a'] = ((df_train['mtr_val_new'] != df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
df_train['R_2b'] = ((df_train['mtr_val_new'] > df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
df_train['R_3a'] = ((df_train['mtr_val_old'] != df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_3b'] = ((df_train['mtr_val_old'] < df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_4a'] = ((df_train['months_num'] != df_train['months_num_calc']) & (df_train['mtr_val_new_prv'].notna()))
df_train['R_4b'] = (((df_train['months_num'] - df_train['months_num_calc']) > 1) & (df_train['mtr_val_new_prv'].notna()))

print(f"Rows breaking Rule 1a: {sum(df_train['R_1a'])} ({round(100 * sum(df_train['R_1a']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2a: {sum(df_train['R_2a'])} ({round(100 * sum(df_train['R_2a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_old_nxt'].isna())}.")
print(f"Rows breaking Rule 3a: {sum(df_train['R_3a'])} ({round(100 * sum(df_train['R_3a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 4a: {sum(df_train['R_4a'])} ({round(100 * sum(df_train['R_4a']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
mask = (df_train['R_1a']) | (df_train['R_2a']) | (df_train['R_2b']) | (df_train['R_3a']) | (df_train['R_3b']) | (df_train['R_4a']) | (df_train['R_4b'])
print(f"Rows breaking any Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")

print(f"Rows breaking Rule 1b: {sum(df_train['R_1b'])} ({round(100 * sum(df_train['R_1b']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2b: {sum(df_train['R_2b'])} ({round(100 * sum(df_train['R_2b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 3b: {sum(df_train['R_3b'])} ({round(100 * sum(df_train['R_3b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
print(f"Rows breaking Rule 4b: {sum(df_train['R_4b'])} ({round(100 * sum(df_train['R_4b']) / len(df_train), 1)}%). Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
mask = (df_train['R_1b']) | (df_train['R_2b']) | (df_train['R_3b']) | (df_train['R_4b'])
print(f"Rows breaking any lax Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")

print(f"Rows removed: {len(df_removed)} ({round(len(df_removed) / (len(df_removed) + len(df_train)), 2)}%).")

Rows breaking Rule 1a: 4909 (0.1%).
Rows breaking Rule 2a: 287445 (6.6%). Excluded NAs: 228999.
Rows breaking Rule 3a: 287445 (6.6%). Excluded NAs: 228999.
Rows breaking Rule 4a: 1879509 (43.2%). Excluded NAs: 228999.
Rows breaking any Rule: 2027663 (46.6%).
Rows breaking Rule 1b: 4579 (0.1%).
Rows breaking Rule 2b: 1773 (0.0%). Excluded NAs: 228999.
Rows breaking Rule 3b: 1773 (0.0%). Excluded NAs: 228999.
Rows breaking Rule 4b: 2957 (0.1%). Excluded NAs: 228999.
Rows breaking any lax Rule: 9866 (0.2%).
Rows removed: 122309 (0.03%).


In [80]:
df_temp = df_train[df_train['R_1b'] == True]
#df_temp['temp_flag'] = df_temp['usage_n_calc'].div(df_temp['usage_n'])
#mask = (df_temp['usage_n_calc'] != 0) & ((df_temp['usage_n'] - df_temp['usage_n_calc']).abs() < 3)
#df_temp[mask][['client_id', 'mtr_tariff', 'mtr_id', 'mtr_status', 'mtr_code', 'mtr_notes', 'mtr_coef', 'usage_N', 'usage_n', 'mtr_val_old', 'mtr_val_new', 'usage_n_calc', 'mtr_val_new_calc', 'temp_flag']]

In [62]:
#df_train_save, df_removed_save = df_train.copy(), df_removed.copy()
df_train, df_removed = df_train_save.copy(), df_removed_save.copy()

##### Retesting 'usage_aux':
Since some:
* Expected values must

In [ ]:
mask = df_train['usage_aux_flag'] != 0 # Those that had overlapping mtr_tariff == 9
df_train['temp_flag'] = np.nan
df_train.loc[mask, 'temp_flag'] = df_train.loc[mask, 'usage_n_calc'] - df_train.loc[mask, 'usage_n']
mask_2 = (df_train['temp_flag'] - df_train['usage_aux']) # Residual delta after making usage of 'usage_aux'
mask_2 = (((mask_2.abs() * 4) < (df_train['temp_flag']).abs()) |
          ((mask_2.abs() < df_train['temp_flag'].abs()) & (mask_2 > -10)))
print(f"Rows where \'usage_aux\' is rejected / rows it was present: {sum(mask & ~mask_2)} / {sum(mask)}.")
#df_train.loc[(mask & ~mask_2), 'usage_aux_flag'] = -1 # These are flagged as rejected, although they are retained for now
df_temp = df_train[(mask & ~mask_2)]
df_temp

Rows where 'usage_aux' is rejected / rows it was present: 1499 / 5644.


,client_id,invoice_date,mtr_tariff,mtr_id,mtr_status,mtr_code,mtr_notes,mtr_coef,usage_1,usage_2,...,mtr_stuck,usage_missing,mtr_reset,temp_flag_2,mtr_max_digit,monthly_usage_grp_median,mtr_old_rollover,mtr_new_rollover,mtr_old_adj_quant,mtr_new_adj_quant
8437,train_Client_100244,2014-05-15,11,630735,0,413,8,1.0,400,200,...,0,0,0,0.0,100000.0,125.750,0,0,0.0,0.0
17318,train_Client_100479,2017-02-14,11,21557018982,0,207,9,1.0,267,0,...,0,1,0,0.0,10000.0,46.875,0,0,0.0,0.0
17319,train_Client_100479,2017-06-09,11,21557018982,0,207,9,1.0,202,0,...,0,1,0,0.0,10000.0,46.875,0,0,0.0,0.0
17321,train_Client_100479,2018-02-13,11,21557018982,0,207,9,1.0,84,0,...,0,1,0,0.0,10000.0,46.875,0,0,0.0,0.0
17322,train_Client_100479,2018-06-06,11,21557018982,0,207,9,1.0,158,0,...,0,1,0,0.0,10000.0,46.875,0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4366035,train_Client_99816,2015-10-20,11,100099,0,413,9,1.0,400,200,...,0,0,0,0.0,100000.0,86.750,0,0,0.0,0.0
4366040,train_Client_99816,2017-06-13,11,100099,0,413,9,1.0,679,0,...,0,1,0,0.0,100000.0,86.750,0,0,0.0,0.0
4366043,train_Client_99816,2018-06-11,11,100099,0,413,9,1.0,654,0,...,0,1,0,0.0,100000.0,86.750,0,0,0.0,0.0
4366046,train_Client_99816,2019-06-21,11,100099,0,413,9,1.0,551,0,...,0,1,0,0.0,100000.0,86.750,0,0,0.0,0.0


In [92]:
df_temp = df_train[df_train['R_1'] == True]

In [ ]:
# OLD
df_train['R_1'] = ~(df_train['mtr_val_old'] + df_train['usage_n'] == df_train['mtr_val_new'])
df_train['R_2a'] = ~((df_train['mtr_val_new'] == df_train['mtr_val_old_nxt']) | df_train['mtr_val_old_nxt'].isna())
df_train['R_2b'] = ~((df_train['mtr_val_new'] <= df_train['mtr_val_old_nxt']) | df_train['mtr_val_old_nxt'].isna())
df_train['R_3a'] = ~((df_train['mtr_val_old'] == df_train['mtr_val_new_prv']) | df_train['mtr_val_new_prv'].isna())
df_train['R_3b'] = ~((df_train['mtr_val_old'] >= df_train['mtr_val_new_prv']) | df_train['mtr_val_new_prv'].isna())
df_train['R_4a'] = ~(df_train['months_num'] == df_train['months_num_calc'])
df_train['R_4b'] = ~(df_train['months_num'] - df_train['months_num_calc'] < 3)

print(f"Rows breaking Rule 1: {sum(df_train['R_1'])} ({round(100 * sum(df_train['R_1']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2a: {sum(df_train['R_2a'])} ({round(100 * sum(df_train['R_2a']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 2b: {sum(df_train['R_2b'])} ({round(100 * sum(df_train['R_2b']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 3a: {sum(df_train['R_3a'])} ({round(100 * sum(df_train['R_3a']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 3b: {sum(df_train['R_3b'])} ({round(100 * sum(df_train['R_3b']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 4a: {sum(df_train['R_4a'])} ({round(100 * sum(df_train['R_4a']) / len(df_train), 1)}%).")
print(f"Rows breaking Rule 4b: {sum(df_train['R_4b'])} ({round(100 * sum(df_train['R_4b']) / len(df_train), 1)}%).")
mask = (df_train['R_1']) | (df_train['R_2a']) | (df_train['R_2b']) | (df_train['R_3a']) | (df_train['R_3b']) | (df_train['R_4a']) | (df_train['R_4b'])
print(f"Rows breaking any Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")
mask = (df_train['R_1']) | (df_train['R_2b']) | (df_train['R_3b']) | (df_train['R_4b'])
print(f"Rows breaking any lax Rule: {sum(mask)} ({round(100 * sum(mask) / len(df_train), 1)}%).")
print(f"Rows removed: {len(df_removed)} ({round(len(df_removed) / (len(df_removed) + len(df_train)), 2)}%).")

Rows breaking Rule 1: 7848 (0.2%).
Rows breaking Rule 2a: 285702 (6.7%).
Rows breaking Rule 2b: 0 (0.0%).
Rows breaking Rule 3a: 285702 (6.7%).
Rows breaking Rule 3b: 0 (0.0%).
Rows breaking Rule 4a: 559099 (13.1%).
Rows breaking Rule 4b: 224442 (5.3%).
Rows breaking any Rule: 777241 (18.2%).
Rows breaking any lax Rule: 231350 (5.4%).
Rows removed: 208062 (0.05%).


In [79]:
mask = df_train['usage_n_calc'] != df_train['usage_n']
mask_2 = (df_train['mtr_val_old'] + df_train['usage_n']) != df_train['mtr_val_new']

print(sum(mask))
print(sum(mask_2))

df_temp = df_train[(mask ^ mask_2)]

9456
9456


In [74]:
print(sum(df_train['mtr_val_old_nxt'].isna()))
print(sum((df_train['mtr_val_new'] == df_train['mtr_val_old_nxt'])))

print(sum((df_train['mtr_val_new'] == df_train['mtr_val_old_nxt']) | (df_train['mtr_val_old_nxt'].isna())))
print(sum((df_train['mtr_val_new'] != df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna())))

print(sum(df_train['mtr_val_old_nxt'].notna()))
print(sum(df_train['mtr_val_old_nxt'].isna() | df_train['mtr_val_old_nxt'].notna()))
print(len(df_train['mtr_val_old_nxt']))

228999
3839479
4068478
285962
4125441
4354440
4354440


In [ ]:
    mask = ((df_train['mtr_val_old'] < df_train['mtr_val_new_prv']) & (df_train['mtr_val_new_prv'].notna()))
    df_train['mtr_flag_bkd'] = mask.astype(int)
    print(f"Rows where meter readings seem out of order looking backwards / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['mtr_val_new_prv'].isna())}.")
    mask = ((df_train['mtr_val_new'] > df_train['mtr_val_old_nxt']) & (df_train['mtr_val_old_nxt'].notna()))
    df_train['mtr_flag_fwd'] = mask.astype(int)
    print(f"Rows where meter readings seem out of order looking forwards / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['mtr_val_old_nxt'].isna())}.")
    mask = ((df_train['invoice_date_prv_calc'] + pd.to_timedelta(30.5, unit='days') < df_train['invoice_date_prv']) & (df_train['invoice_date_prv_calc'].notna()))
    df_train['date_flag'] = mask.astype(int)
    print(f"Rows where invoice spans seem to overlap based on \'months_num\' / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['months_num'].isna())}.")
    mask = ((df_train['months_num_calc'] < df_train['months_num']) & (df_train['invoice_date_prv_calc'].notna()))
    df_train.loc[mask, 'date_flag'] = 1
    print(f"Rows where invoice spans seem to overlap based on \'invoice_date\'s / Total Rows: {sum(mask)} / {rows}. Excluded NAs: {sum(df_train['months_num'].isna())}.")